# DB 적재용 더미 데이터 생성

In [1]:
from faker import Faker
from sdv.metadata import MultiTableMetadata
from sdv.metadata import Metadata
from sdv.single_table import GaussianCopulaSynthesizer

import os
import re
import random
from datetime import timedelta, datetime, timezone
from collections import defaultdict, Counter
import uuid

import hashlib
import os
import base64

import numpy as np
import pandas as pd

import dask
from dask import delayed
import time
from tqdm import tqdm
import threading

import bcrypt

## 1. 고객 패스워드 복호화

In [2]:
def make_password(password, cost_factor=12):
    """
    bcrypt를 사용하여 암호 해시를 생성합니다.

    • password: 평문 문자열 (UTF-8)
    • cost_factor: bcrypt의 작업량 계수 (cost factor). 
                   숫자가 클수록 더 많은 계산이 필요하여 안전하지만 느려집니다. 
                   일반적으로 12-14가 권장됩니다.
    """
    # 비밀번호를 UTF-8 바이트로 인코딩합니다.
    password_bytes = password.encode('utf-8')
    
    # 지정된 cost_factor로 솔트(salt)를 생성합니다.
    salt = bcrypt.gensalt(rounds=cost_factor)
    
    # 비밀번호와 솔트를 해싱합니다.
    hash_bytes = bcrypt.hashpw(password_bytes, salt)
    
    # 최종 해시 문자열을 반환합니다 (UTF-8로 디코딩).
    # 이 문자열 안에는 알고리즘, cost factor, salt, 해시 값이 모두 포함됩니다.
    return hash_bytes.decode('utf-8')

## 2. 참조 필요 없는 데이터 생성

In [3]:
# ─── 상수 정의 ───
NUM_USERS       = 80000
# NUM_PROMOTIONS  = 5
# NUM_ORDERS      = 50000
# NUM_CARTS       = 50000

fake = Faker('ko_KR')

### 2-1. Recipes

In [9]:
# Recipes
recipes_df = pd.read_csv("data/recipes.csv")


In [38]:
recipes_24_df = pd.read_csv("data/TB_RECIPE_SEARCH_241226.csv", encoding='utf-8-sig')

In [50]:
recipes_24_df.head(3)

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT,RCP_IMG_URL
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,7016814,#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육,된장수육,kstencil,강철새잎,1396,0,1,삶기,술안주,돼지고기,메인반찬,수육용 삼겹살을 사다가 된장과 술을 넣고 일상먹거리 수육한접시를 만들어 주었습니다....,[재료] 돼지고기 수육용삼겹살500g| 된장1.5큰술| 술4큰술| ...,2인분,아무나,2시간이내,20240101002917,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [39]:
np.min(recipes_24_df["FIRST_REG_DT"])

np.int64(20240101000857)

In [40]:
np.max(recipes_24_df["FIRST_REG_DT"])

np.int64(20241226235311)

In [63]:
df_20231130 = pd.read_csv("data/TB_RECIPE_SEARCH-20231130.csv", 
                        encoding='cp949', 
                        encoding_errors='ignore')

print("✅ 한글 깨짐 문제 해결 완료!")
print(f"데이터 형태: {df_20231130.shape}")
print(f"컬럼명: {list(df_20231130.columns)}")
print("\n레시피 제목 샘플 (한글 정상 표시):")
print(df_20231130['RCP_TTL'].head(10).to_list())
print("\n요리 종류 샘플:")

✅ 한글 깨짐 문제 해결 완료!
데이터 형태: (184991, 18)
컬럼명: ['RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM', 'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN', 'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT']

레시피 제목 샘플 (한글 정상 표시):
['어묵김말이', '두부에 꼬리가 달렸어요!! ', '입안에서 톡톡톡 ', '★현미호두죽', '부들부들 보들보들 북어갈비♥', '토마토스파게티♥', '표고버섯탕수', '달콤한 마늘향이 가득해~ 갈릭치킨♥', '바삭바삭 양파링 튀김 만들기', '참치 삼각김밥 삼총사']

요리 종류 샘플:


In [62]:
len(recipes_24_df)

184991

In [53]:
recipes_24_df.head(3)

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT,RCP_IMG_URL
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,7016814,#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육,된장수육,kstencil,강철새잎,1396,0,1,삶기,술안주,돼지고기,메인반찬,수육용 삼겹살을 사다가 된장과 술을 넣고 일상먹거리 수육한접시를 만들어 주었습니다....,[재료] 돼지고기 수육용삼겹살500g| 된장1.5큰술| 술4큰술| ...,2인분,아무나,2시간이내,20240101002917,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [67]:
df_20231130.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184991 entries, 0 to 184990
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   RCP_SNO           184991 non-null  int64 
 1   RCP_TTL           184991 non-null  object
 2   CKG_NM            184916 non-null  object
 3   RGTR_ID           184991 non-null  object
 4   RGTR_NM           184975 non-null  object
 5   INQ_CNT           184991 non-null  int64 
 6   RCMM_CNT          184991 non-null  int64 
 7   SRAP_CNT          184991 non-null  int64 
 8   CKG_MTH_ACTO_NM   184990 non-null  object
 9   CKG_STA_ACTO_NM   183951 non-null  object
 10  CKG_MTRL_ACTO_NM  184989 non-null  object
 11  CKG_KND_ACTO_NM   184990 non-null  object
 12  CKG_IPDC          184079 non-null  object
 13  CKG_MTRL_CN       183996 non-null  object
 14  CKG_INBUN_NM      180189 non-null  object
 15  CKG_DODF_NM       184714 non-null  object
 16  CKG_TIME_NM       168583 non-null  obj

In [64]:
print(np.min(df_20231130["FIRST_REG_DT"]))
print(np.max(df_20231130["FIRST_REG_DT"]))

20070402131403
20231130235050


In [72]:
total_recipes = pd.concat([recipes_24_df, df_20231130], ignore_index=True)

In [74]:
total_recipes.head(3)

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT,RCP_IMG_URL
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,7016814,#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육,된장수육,kstencil,강철새잎,1396,0,1,삶기,술안주,돼지고기,메인반찬,수육용 삼겹살을 사다가 된장과 술을 넣고 일상먹거리 수육한접시를 만들어 주었습니다....,[재료] 돼지고기 수육용삼겹살500g| 된장1.5큰술| 술4큰술| ...,2인분,아무나,2시간이내,20240101002917,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [77]:
total_recipes["CKG_MTRL_ACTO_NM"].value_counts()

CKG_MTRL_ACTO_NM
채소류       59257
해물류       22563
가공식품류     22140
밀가루       15287
돼지고기      13020
달걀/유제품    12352
기타        10853
소고기        9815
닭고기        8568
콩/견과류      6977
쌀          6973
과일류        6661
건어물류       5733
버섯류        3259
육류         2425
곡류         2296
Name: count, dtype: int64

In [82]:
recipes_df["situation_type"].value_counts()

situation_type
DAILY         863
QUICK         629
SNACK         573
GUEST         521
DRINK         421
NUTRITION     296
DIET          262
ETC           161
LUNCHBOX      147
HOLIDAY       129
STYLING        57
LATE_NIGHT     50
BABY           41
HANGOVER       33
ALL             1
Name: count, dtype: int64

In [75]:
print(np.min(total_recipes["FIRST_REG_DT"]))
print(np.max(total_recipes["FIRST_REG_DT"]))

20070402131403
20241226235311


In [76]:
recipes_df.head(2)

,id,created_at,created_by,modified_at,modified_by,content,cooking_time,deleted_at,difficulty,dish_type,file_path,image_url,ingredient_type,is_deleted,key_name,method_type,object_name,situation_type,title,user_id
0,1,2025-05-15 01:07:02.662,158.0,NaN,NaN,[소개]\n\n\n[재료]\n\n\n[조리 순서]\n,30,NaN,초급,NaN,NaN,https://recipe1.ezmember.co.kr/cache/recipe/20...,NaN,False,NaN,SEASONED_MIX,NaN,DRINK,오우 매콤해서 더욱 입맛 도는 골뱅이 묻침...,158
1,2,2025-05-15 01:07:02.662,65.0,NaN,NaN,[소개]\n여러가지 재료들 넣어서 슥슥 버무려서 입에 착 달라붙는 맛있는 참치 샌드...,15,NaN,아무나,NaN,NaN,https://recipe1.ezmember.co.kr/cache/recipe/20...,PROCESSED,False,NaN,SEASONED_MIX,NaN,NaN,매실장아찌를 넣은 참치 샌드위치,65


In [100]:
# 방법 1: 결과를 새로운 변수에 저장
total_recipes_renamed = total_recipes.rename(columns={
    "RCP_SNO": "id", 
    "RCP_TTL": "title", 
    "CKG_NM": "name", 
    "RGTR_ID": "user_name",
    "INQ_CNT": "view_cnt", 
    "RCMM_CNT": "recommend_cnt", 
    "SRAP_CNT": "scrap_cnt",
    "CKG_MTH_ACTO_NM": "method_type", 
    "CKG_STA_ACTO_NM": "situation_type", 
    "CKG_MTRL_ACTO_NM": "ingredient_type", 
    "CKG_KND_ACTO_NM": "dish_type",
    "CKG_IPDC": "content", 
    "CKG_MTRL_CN": "ingredient_list", 
    "CKG_INBUN_NM": "serving",
    "CKG_DODF_NM": "difficulty", 
    "CKG_TIME_NM": "cooking_time", 
    "FIRST_REG_DT": "created_at",
    "RCP_IMG_URL": "image_url"
})

print("변경된 컬럼명:")
print(list(total_recipes_renamed.columns))

변경된 컬럼명:
['id', 'title', 'name', 'user_name', 'RGTR_NM', 'view_cnt', 'recommend_cnt', 'scrap_cnt', 'method_type', 'situation_type', 'ingredient_type', 'category', 'content', 'ingredient_list', 'serving', 'difficulty', 'cooking_time', 'created_at', 'image_url']


In [101]:
total_recipes_renamed.head(1)

,id,title,name,user_name,RGTR_NM,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,category,content,ingredient_list,serving,difficulty,cooking_time,created_at,image_url
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [ ]:
recipes_df["dish_type"].value_counts()

In [134]:
# 사용자 이름에 따라 고유한 user_id 생성
print("🔍 현재 사용자 이름 분석")
print("="*50)

# 현재 user_name 컬럼 확인 (기존 user_name는 문자열 ID)
print("현재 user_name 컬럼의 데이터 타입:", total_recipes_renamed['user_name'].dtype)
print("현재 user_name 샘플:", total_recipes_renamed['user_name'].head().tolist())
print()

# RGTR_NM (사용자 이름) 컬럼 확인
user_name_col = None
for col in total_recipes_renamed.columns:
    if 'RGTR_NM' in col or 'user_name' in col.lower() or 'name' in col.lower():
        if col != 'name':  # 레시피 이름이 아닌 컬럼
            user_name_col = col
            break

if user_name_col is None:
    print("⚠️ 사용자 이름 컬럼을 찾을 수 없습니다.")
    print("사용 가능한 컬럼:", list(total_recipes_renamed.columns))
else:
    print(f"✅ 사용자 이름 컬럼: {user_name_col}")
    print(f"고유 사용자 수: {total_recipes_renamed[user_name_col].nunique()}")
    print(f"전체 레코드 수: {len(total_recipes_renamed)}")
    print()
    
    # 사용자 이름별 레코드 수 확인
    print("사용자별 레시피 수 (상위 10명):")
    print(total_recipes_renamed[user_name_col].value_counts().head(10))
    print()
    
    # 고유한 user_id 생성 (사용자 이름 기반)
    print("📋 고유한 user_id 생성 중...")
    
    # 방법 1: pandas의 Categorical 사용
    unique_users = total_recipes_renamed[user_name_col].unique()
    user_mapping = {user: idx + 1 for idx, user in enumerate(unique_users)}
    
    # 새로운 user_id 컬럼 생성
    total_recipes_renamed['user_id'] = total_recipes_renamed[user_name_col].map(user_mapping)
    
    print(f"✅ 고유한 user_id 생성 완료!")
    print(f"생성된 user_id 범위: 1 ~ {total_recipes_renamed['user_id'].max()}")
    print()
    
    # 결과 확인
    print("🔍 생성된 user_id 확인:")
    sample_data = total_recipes_renamed[[user_name_col, 'user_id', 'user_id']].head(10)
    print(sample_data)
    print()
    
    # 중복 확인
    print("🔍 같은 사용자 이름이 같은 user_id를 가지는지 확인:")
    check_df = total_recipes_renamed.groupby(user_name_col)['user_id'].nunique()
    duplicates = check_df[check_df > 1]
    
    if len(duplicates) == 0:
        print("✅ 모든 사용자 이름이 고유한 user_id를 가집니다!")
    else:
        print(f"⚠️ 문제가 있는 사용자: {len(duplicates)}명")
        print(duplicates.head())
        
    # 사용자 매핑 테이블 일부 출력
    print("\n📊 사용자 매핑 테이블 (처음 10개):")
    mapping_df = pd.DataFrame(list(user_mapping.items()), columns=['user_name', 'user_id'])
    print(mapping_df.head(10))

🔍 현재 사용자 이름 분석
현재 user_name 컬럼의 데이터 타입: object
현재 user_name 샘플: ['ranch6356', 'kstencil', '87771622', 'aeaelove', 'aeaelove']

✅ 사용자 이름 컬럼: user_name
고유 사용자 수: 6450
전체 레코드 수: 208183

사용자별 레시피 수 (상위 10명):
user_name
gdubu33         5170
kstencil        3473
achim100        3423
hskim4127       3101
32061214        3059
89234500        2796
limsu11         2789
37279613        2379
62299079        2329
jylhee070467    2265
Name: count, dtype: int64

📋 고유한 user_id 생성 중...
✅ 고유한 user_id 생성 완료!
생성된 user_id 범위: 1 ~ 6450

🔍 생성된 user_id 확인:
   user_name  user_id  user_id
0  ranch6356        1        1
1   kstencil        2        2
2   87771622        3        3
3   aeaelove        4        4
4   aeaelove        4        4
5   kstencil        2        2
6   77721145        5        5
7   77721145        5        5
8   77721145        5        5
9   77721145        5        5

🔍 같은 사용자 이름이 같은 user_id를 가지는지 확인:
✅ 모든 사용자 이름이 고유한 user_id를 가집니다!

📊 사용자 매핑 테이블 (처음 10개):
   user_name  user_id
0  ranch

In [135]:
total_recipes_renamed["created_by"] = total_recipes_renamed["user_id"]
total_recipes_renamed["modified_at"] = total_recipes_renamed["created_at"]
total_recipes_renamed["modified_by"] = total_recipes_renamed["user_id"]

# 비어 있는 값
total_recipes_renamed['is_deleted'] = False

In [136]:
total_recipes_renamed.head(1)


,id,title,name,user_name,RGTR_NM,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,dish_type,content,ingredient_list,serving,difficulty,cooking_time,created_at,image_url,user_id,modified_at,modified_by,created_by,is_deleted
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...,1,20240101000857,1,1,False


In [137]:
# recipes_df와 total_recipes_renamed 컬럼 비교
print("📊 데이터프레임 컬럼 비교 분석")
print("="*60)

# 각 데이터프레임의 컬럼 확인
print(f"📋 recipes_df 컬럼 개수: {len(recipes_df.columns)}")
print(f"📋 total_recipes_renamed 컬럼 개수: {len(total_recipes_renamed.columns)}")
print()

# 컬럼을 집합으로 변환
recipes_df_cols = set(recipes_df.columns)
total_recipes_renamed_cols = set(total_recipes_renamed.columns)

print("🔍 recipes_df의 컬럼:")
print(sorted(recipes_df_cols))
print()

print("🔍 total_recipes_renamed의 컬럼:")
print(sorted(total_recipes_renamed_cols))
print()

# total_recipes_renamed에 없는 컬럼들 (recipes_df에만 있는 컬럼들)
missing_in_renamed = recipes_df_cols - total_recipes_renamed_cols
print("❌ total_recipes_renamed에 없는 컬럼들 (recipes_df에만 있음):")
if missing_in_renamed:
    for col in sorted(missing_in_renamed):
        print(f"  - {col}")
        # 해당 컬럼의 샘플 데이터 확인
        sample_values = recipes_df[col].dropna().head(3).tolist()
        print(f"    샘플 데이터: {sample_values}")
else:
    print("  없음 - 모든 컬럼이 total_recipes_renamed에 존재합니다!")
print()

# recipes_df에 없는 컬럼들 (total_recipes_renamed에만 있는 컬럼들)
missing_in_original = total_recipes_renamed_cols - recipes_df_cols
print("➕ recipes_df에 없는 컬럼들 (total_recipes_renamed에만 있음):")
if missing_in_original:
    for col in sorted(missing_in_original):
        print(f"  - {col}")
        # 해당 컬럼의 샘플 데이터 확인
        sample_values = total_recipes_renamed[col].dropna().head(3).tolist()
        print(f"    샘플 데이터: {sample_values}")
else:
    print("  없음")
print()

# 공통 컬럼들
common_cols = recipes_df_cols & total_recipes_renamed_cols
print(f"🤝 공통 컬럼들 ({len(common_cols)}개):")
print(sorted(common_cols))
print()

# 요약
print("📊 요약:")
print(f"  - recipes_df 전용 컬럼: {len(missing_in_renamed)}개")
print(f"  - total_recipes_renamed 전용 컬럼: {len(missing_in_original)}개")
print(f"  - 공통 컬럼: {len(common_cols)}개")

📊 데이터프레임 컬럼 비교 분석
📋 recipes_df 컬럼 개수: 20
📋 total_recipes_renamed 컬럼 개수: 24

🔍 recipes_df의 컬럼:
['content', 'cooking_time', 'created_at', 'created_by', 'deleted_at', 'difficulty', 'dish_type', 'file_path', 'id', 'image_url', 'ingredient_type', 'is_deleted', 'key_name', 'method_type', 'modified_at', 'modified_by', 'object_name', 'situation_type', 'title', 'user_id']

🔍 total_recipes_renamed의 컬럼:
['RGTR_NM', 'content', 'cooking_time', 'created_at', 'created_by', 'difficulty', 'dish_type', 'id', 'image_url', 'ingredient_list', 'ingredient_type', 'is_deleted', 'method_type', 'modified_at', 'modified_by', 'name', 'recommend_cnt', 'scrap_cnt', 'serving', 'situation_type', 'title', 'user_id', 'user_name', 'view_cnt']

❌ total_recipes_renamed에 없는 컬럼들 (recipes_df에만 있음):
  - deleted_at
    샘플 데이터: ['2025-05-22 22:30:00.046', '2025-05-23 09:29:42.693', '2025-06-17 13:31:38.348']
  - file_path
    샘플 데이터: []
  - key_name
    샘플 데이터: []
  - object_name
    샘플 데이터: []

➕ recipes_df에 없는 컬럼들 (total_reci

In [132]:
recipes_df["object_name"].value_counts()

,id,title,name,user_name,RGTR_NM,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,dish_type,content,ingredient_list,serving,difficulty,cooking_time,created_at,image_url,user_id,modified_at,modified_by,created_by,is_deleted
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...,1,20240101000857,1,1,False


📊 데이터프레임 컬럼 비교 분석
📋 recipes_df 컬럼 개수: 20
📋 total_recipes_renamed 컬럼 개수: 24

🔍 recipes_df의 컬럼:
['content', 'cooking_time', 'created_at', 'created_by', 'deleted_at', 'difficulty', 'dish_type', 'file_path', 'id', 'image_url', 'ingredient_type', 'is_deleted', 'key_name', 'method_type', 'modified_at', 'modified_by', 'object_name', 'situation_type', 'title', 'user_id']

🔍 total_recipes_renamed의 컬럼:
['RGTR_NM', 'content', 'cooking_time', 'created_at', 'created_by', 'difficulty', 'dish_type', 'id', 'image_url', 'ingredient_list', 'ingredient_type', 'is_deleted', 'method_type', 'modified_at', 'modified_by', 'name', 'recommend_cnt', 'scrap_cnt', 'serving', 'situation_type', 'title', 'user_id', 'user_name', 'view_cnt']

❌ total_recipes_renamed에 없는 컬럼들 (recipes_df에만 있음):
  - deleted_at
    샘플 데이터: ['2025-05-22 22:30:00.046', '2025-05-23 09:29:42.693', '2025-06-17 13:31:38.348']
  - file_path
    샘플 데이터: []
  - key_name
    샘플 데이터: []
  - object_name
    샘플 데이터: []

➕ recipes_df에 없는 컬럼들 (total_reci

In [22]:
recipes_df["title"].loc[0]

'오우 매콤해서 더욱 입맛 도는 골뱅이 묻침...'

In [20]:
recipes_df["image_url"].loc[0]

'https://recipe1.ezmember.co.kr/cache/recipe/2017/07/25/72024cd19b9c255739404870dbe25fd2.jpg'

In [14]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10008 entries, 0 to 10007
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10008 non-null  int64  
 1   created_at       10008 non-null  object 
 2   created_by       9992 non-null   float64
 3   modified_at      16 non-null     object 
 4   modified_by      0 non-null      float64
 5   content          10008 non-null  object 
 6   cooking_time     9285 non-null   object 
 7   deleted_at       3 non-null      object 
 8   difficulty       9531 non-null   object 
 9   dish_type        5225 non-null   object 
 10  file_path        0 non-null      float64
 11  image_url        9978 non-null   object 
 12  ingredient_type  5421 non-null   object 
 13  is_deleted       10008 non-null  bool   
 14  key_name         0 non-null      float64
 15  method_type      5662 non-null   object 
 16  object_name      0 non-null      float64
 17  situation_ty

In [12]:
recipes_df["dish_type"].value_counts()

dish_type
BANCHAN            650
MAIN_DISH          640
RICE_PORRIDGE      506
SOUP               504
BREAD              451
NOODLE_DUMPLING    367
ETC                366
DESSERT            362
KIMCHI_SAUCE       270
WESTERN            193
SALAD              180
STEW               174
BEVERAGE           159
SNACK              132
SEASONING          122
FUSION             105
SOUP_WESTERN        43
ALL                  1
Name: count, dtype: int64

In [15]:
recipes_df["ingredient_type"].value_counts()

ingredient_type
VEGETABLE    711
SEAFOOD      543
PROCESSED    541
FLOUR        483
PORK         470
EGG_DAIRY    405
CHICKEN      371
ETC          348
BEEF         307
FRUIT        273
BEAN_NUT     260
DRIED        227
RICE         220
GRAIN         91
MEAT          91
MUSHROOM      79
ALL            1
Name: count, dtype: int64

In [16]:
recipes_df["situation_type"].value_counts()

situation_type
DAILY         863
QUICK         629
SNACK         573
GUEST         521
DRINK         421
NUTRITION     296
DIET          262
ETC           161
LUNCHBOX      147
HOLIDAY       129
STYLING        57
LATE_NIGHT     50
BABY           41
HANGOVER       33
ALL             1
Name: count, dtype: int64

In [17]:
def unified_shuffle_masking(text, seed=42, force_change=True):
    """
    통일된 순서 변경 마스킹 함수
    - 모든 길이의 단어가 변경되도록 보장
    - 같은 입력은 항상 같은 출력 보장 (일관성)
    - 한글/영문 모두 지원
    """
    if pd.isna(text) or text == '':
        return text
    
    # 입력값 기반으로 시드 생성 (일관성 보장)
    text_seed = hash(str(text) + str(seed)) % (2**32)
    random.seed(text_seed)
    
    result = ""
    # 한글, 영문, 숫자, 특수문자를 분리하여 처리
    tokens = re.findall(r'[가-힣]+|[A-Za-z]+|[0-9]+|[^가-힣A-Za-z0-9\s]+|\s+', str(text))
    
    for token in tokens:
        if re.match(r'[가-힣]+|[A-Za-z]+', token):  # 한글 또는 영문 단어
            if len(token) == 1:
                # 1글자는 반복하여 변경 보장
                result += token + token
            elif len(token) == 2:
                # 2글자는 순서 바꾸기
                result += token[1] + token[0]
            elif len(token) == 3:
                # 3글자는 전체 섞기 (원본과 다를 때까지)
                chars = list(token)
                attempts = 0
                while ''.join(chars) == token and attempts < 10:
                    random.shuffle(chars)
                    attempts += 1
                result += ''.join(chars)
            else:
                # 4글자 이상은 첫/끝 고정, 중간 섞기
                first_char = token[0]
                last_char = token[-1]
                middle_chars = list(token[1:-1])
                
                # 중간 부분이 변경되도록 보장
                original_middle = ''.join(middle_chars)
                attempts = 0
                while ''.join(middle_chars) == original_middle and attempts < 10:
                    random.shuffle(middle_chars)
                    attempts += 1
                
                result += first_char + ''.join(middle_chars) + last_char
        else:
            # 숫자, 특수문자, 공백은 그대로 유지
            result += token
    
    # 최종 검증: 원본과 동일하면 마스킹 표시 추가
    if force_change and result == str(text):
        result = str(text) + "_M"
    
    return result

def apply_unified_masking():
    """
    통일된 마스킹 함수를 사용하여 product와 brand 데이터 마스킹
    """
    print("=== 통일된 마스킹 적용 ===")
    
    # 이미 마스킹된 데이터가 있다면 원본으로 복원
    if 'name_original' in product.columns:
        product['name'] = product['name_original']
    else:
        product['name_original'] = product['name']
    
    if 'brand_kor_original' in brand.columns:
        brand['brand_kor'] = brand['brand_kor_original']
        brand['brand_eng'] = brand['brand_eng_original']
    else:
        brand['brand_kor_original'] = brand['brand_kor']
        brand['brand_eng_original'] = brand['brand_eng']
    
    # 통일된 마스킹 적용
    print("1. Product name 마스킹 중...")
    product['name'] = product['name_original'].apply(
        lambda x: unified_shuffle_masking(x, seed=42)
    )
    
    print("2. Brand 마스킹 중...")
    brand['brand_kor'] = brand['brand_kor_original'].apply(
        lambda x: unified_shuffle_masking(x, seed=42)
    )
    brand['brand_eng'] = brand['brand_eng_original'].apply(
        lambda x: unified_shuffle_masking(x, seed=42)
    )
    
    print("3. 마스킹 완료!")
    
    # 결과 확인
    print("\n=== 마스킹 결과 확인 ===")
    print("Product name 마스킹 결과 (상위 5개):")
    comparison_df = pd.DataFrame({
        'original': product['name_original'].head(5),
        'masked': product['name'].head(5)
    })
    print(comparison_df.to_string(index=False))
    
    print("\nBrand 마스킹 결과 (상위 5개):")    
    brand_comparison = pd.DataFrame({
        'brand_kor_original': brand['brand_kor_original'].head(5),
        'brand_kor_masked': brand['brand_kor'].head(5),
        'brand_eng_original': brand['brand_eng_original'].head(5),
        'brand_eng_masked': brand['brand_eng'].head(5)
    })
    print(brand_comparison.to_string(index=False))

def validate_unified_masking():
    """
    통일된 마스킹의 품질 검증
    """
    print("\n=== 마스킹 품질 검증 ===")
    
    # 1. 길이 변화 확인 (1글자 단어는 2배가 됨)
    length_issues = 0
    for i in range(len(product)):
        orig_len = len(str(product['name_original'].iloc[i])) if pd.notna(product['name_original'].iloc[i]) else 0
        mask_len = len(str(product['name'].iloc[i])) if pd.notna(product['name'].iloc[i]) else 0
        
        # 1글자 단어가 포함된 경우 길이가 늘어날 수 있음
        if orig_len > 0 and mask_len < orig_len:
            length_issues += 1
    
    print(f"길이 관련 이슈: {length_issues}개")
    
    # 2. 변경 효과 확인
    identical_count = (product['name_original'] == product['name']).sum()
    total_count = len(product)
    change_rate = (total_count - identical_count) / total_count * 100
    
    print(f"전체 데이터: {total_count}개")
    print(f"변경된 데이터: {total_count - identical_count}개")
    print(f"변경률: {change_rate:.1f}%")
    print(f"동일한 데이터: {identical_count}개 ({identical_count/total_count*100:.1f}%)")
    
    # 3. 일관성 확인
    consistency_check = True
    value_counts = product['name_original'].value_counts()
    
    for orig_val in value_counts.index[:10]:
        if value_counts[orig_val] > 1:
            masked_values = product[product['name_original'] == orig_val]['name'].unique()
            if len(masked_values) > 1:
                consistency_check = False
                print(f"일관성 실패: '{orig_val}' -> {masked_values}")
                break
    
    print(f"일관성 유지: {'✓' if consistency_check else '✗'}")
    
    # 4. 검증 기준
    threshold = total_count * 0.05
    different_enough = identical_count < threshold
    
    print(f"\n검증 결과:")
    print(f"  허용 동일 값 수 (5%): {threshold:.0f}개")
    print(f"  실제 동일 값 수: {identical_count}개")
    print(f"  different_enough: {'✓' if different_enough else '✗'}")
    
    return {
        'change_rate': change_rate,
        'consistency': consistency_check,
        'different_enough': different_enough,
        'identical_count': identical_count
    }

# 통일된 마스킹 실행
print("통일된 순서 변경 마스킹을 시작합니다...")
apply_unified_masking()

# 검증 실행
unified_validation = validate_unified_masking()

# 최종 결과 요약
print(f"\n=== 최종 결과 요약 ===")
print(f"✓ Product 테이블: {len(product)}개 레코드 마스킹 완료")
print(f"✓ Brand 테이블: {len(brand)}개 레코드 마스킹 완료")
print(f"✓ 변경률: {unified_validation['change_rate']:.1f}%")

all_passed = all(unified_validation.values()) if isinstance(unified_validation['change_rate'], bool) else \
             unified_validation['consistency'] and unified_validation['different_enough']
print(f"✓ 품질 검증: {'모두 통과' if all_passed else '일부 실패'}")

if not all_passed:
    print("검증 실패 항목:")
    if not unified_validation['consistency']:
        print("  - consistency")
    if not unified_validation['different_enough']:
        print("  - different_enough")

통일된 순서 변경 마스킹을 시작합니다...
=== 통일된 마스킹 적용 ===


NameError: name 'product' is not defined

In [9]:
product.head(3)

,id,category_code,major_category,sub_category,brand_id,gender,name,discount,discounted_price,price,rank,like_count,view_count,purchase_count,created_at,updated_at,img_url,name_original
0,2353497,2019,outer,anoraks,379,U,LO209_러클닝럽 랙트 켓자_이네비,54,59000,129000,1,66779,61399,16070,2025-05-17 10:32:18,2025-05-17 10:32:18,https://highfive-goorm-images.s3.amazonaws.com...,OL209_러닝클럽 트랙 자켓_네이비
1,3790860,2001,outer,blousons_ma_1,43,M,릴스렉드 미멀니 킷재 [랙블],50,44990,89900,1,9623,33280,7058,2025-05-17 10:32:18,2025-05-17 10:32:18,https://highfive-goorm-images.s3.amazonaws.com...,릴렉스드 미니멀 재킷 [블랙]
2,3813327,103002,shoes,boots,199,U,810ss 케르마 디모 Cchoaral,0,115000,115000,1,9546,39301,5979,2025-05-17 10:32:18,2025-05-17 10:32:18,https://highfive-goorm-images.s3.amazonaws.com...,810s 마르케 모디 Charcoal


In [10]:
brand.head(3)

,id,brand_kor,brand_eng,like_count,created_at,updated_at,brand_kor_original,brand_eng_original
0,1,앤토토,ttoanwe,21858,2025-05-17 10:32:18,2025-05-17 10:32:18,토앤토,tawntoe
1,2,스쳐케스,sekerhcs,9279,2025-05-17 10:32:18,2025-05-17 10:32:18,스케쳐스,skechers
2,3,아키래클식,aslkiiicsiac,34177,2025-05-17 10:32:18,2025-05-17 10:32:18,아키클래식,akiiiclassic


In [8]:
def convert_to_iso8601_with_utc(dt_obj):
    """
    datetime 객체를 정확한 ISO8601 형태로 변환 (UTC 시간대 포함)
    """
    if pd.isna(dt_obj):
        return None
    
    # datetime 객체인 경우
    if isinstance(dt_obj, datetime):
        # 시간대 정보가 없으면 UTC로 가정
        if dt_obj.tzinfo is None:
            dt_obj = dt_obj.replace(tzinfo=timezone.utc)
        # UTC로 변환 후 ISO 형태로 변환
        return dt_obj.astimezone(timezone.utc).isoformat().replace('+00:00', 'Z')
    
    # 문자열인 경우 datetime으로 파싱 후 변환
    if isinstance(dt_obj, str):
        try:
            parsed_dt = pd.to_datetime(dt_obj)
            if parsed_dt.tzinfo is None:
                parsed_dt = parsed_dt.replace(tzinfo=timezone.utc)
            return parsed_dt.astimezone(timezone.utc).isoformat().replace('+00:00', 'Z')
        except:
            return dt_obj
    
    return dt_obj

In [12]:
# 방법 1: 수동으로 UTC ISO8601 변환 후 저장 (추천)
datetime_cols = ['created_at', 'updated_at']

# product 데이터 저장
product_iso = product.copy()
for col in ['created_at', 'updated_at']:
    if col in product_iso.columns:
        product_iso[col] = product_iso[col].apply(convert_to_iso8601_with_utc)

# brand 데이터 저장
brand_iso = brand.copy()
for col in ['created_at', 'updated_at']:
    if col in brand_iso.columns:
        brand_iso[col] = brand_iso[col].apply(convert_to_iso8601_with_utc)        



### 2-2. User

In [24]:
@delayed
def generate_unique_ids_batch_parallel(batch_size, seed_offset=0):
    """UUID 배치 병렬 생성 - 중복 방지 개선"""
    random.seed(seed_offset)
    batch_ids = []
    
    # UUID는 거의 중복되지 않으므로 단순 생성
    for _ in range(batch_size):
        batch_ids.append(str(uuid.uuid4()))
    
    return batch_ids

@delayed
def generate_unique_accounts_batch_parallel(batch_size, seed_offset=0):
    """계정명 배치 병렬 생성 - 중복 방지 개선"""
    fake_local = Faker('ko_KR')
    fake_local.seed_instance(seed_offset)
    
    batch_accounts = []
    used_accounts = set()
    attempts = 0
    max_attempts = batch_size * 20  # 더 많은 시도 허용
    
    while len(batch_accounts) < batch_size and attempts < max_attempts:
        # 더 유니크한 계정명 생성을 위해 숫자 조합
        base_name = fake_local.user_name()
        account_name = f"{base_name}_{random.randint(1000, 9999)}"
        
        if account_name not in used_accounts:
            batch_accounts.append(account_name)
            used_accounts.add(account_name)
        
        attempts += 1
    
    # 부족한 경우 강제로 유니크한 계정 생성
    while len(batch_accounts) < batch_size:
        account_name = f"user_{seed_offset}_{len(batch_accounts)}_{random.randint(10000, 99999)}"
        batch_accounts.append(account_name)
    
    return batch_accounts

@delayed
def create_user_batch_with_locale(user_ids_batch, accounts_batch, seed_offset=0, locale='ko_KR'):
    """사용자 데이터 배치 생성"""
    fake_local = Faker(locale)
    fake_local.seed_instance(seed_offset)
    random.seed(seed_offset)
    
    batch_users = []
    
    for user_id, account in zip(user_ids_batch, accounts_batch):
        created = fake_local.date_time_between('-1y', 'now')
        raw_pw = fake_local.password(length=12)
        
        # 나이 분포 조정
        if random.random() < 0.8:
            age = random.randint(15, 39)
        else:
            age = random.randint(40, 50)
        
        # 성별 분포 조정
        gender = 'M' if random.random() < 0.8 else 'F'
        
        # updated_at은 created_at 이후로 설정
        updated = created + timedelta(days=random.randint(0, 30))
        
        batch_users.append({
            'id': user_id,
            'account': account,
            'password': make_password(raw_pw),
            'age': age,
            'name': fake_local.name(),
            'gender': gender,
            'created_at': convert_to_iso8601_with_utc(created),
            'updated_at': convert_to_iso8601_with_utc(updated),
            'address': fake_local.address().replace('\n', ' ')
        })
    
    return batch_users

def generate_users_with_dask_optimized(num_users, batch_size=2000, locale='ko_KR'):
    """정확한 개수의 사용자 데이터 생성"""
    print(f"🚀 최적화된 Dask로 정확히 {num_users:,}명의 한국 사용자 데이터 생성 시작...")
    
    # 중복을 고려해서 약간 더 많이 생성
    safety_margin = int(num_users * 0.1)  # 10% 여유분
    target_generation = num_users + safety_margin
    
    # 배치 수 계산
    num_batches = (target_generation + batch_size - 1) // batch_size
    batch_sizes = [batch_size] * (num_batches - 1) + [target_generation - (num_batches - 1) * batch_size]
    
    print(f"📊 여유분 포함 {target_generation:,}개 생성 목표 (실제 필요: {num_users:,}개)")
    print(f"📊 총 {num_batches}개 배치로 병렬 처리 (배치 크기: {batch_size:,})")
    
    # 1단계: ID와 계정 병렬 생성
    print("\n1️⃣ ID 및 계정 병렬 생성...")
    id_tasks = []
    account_tasks = []
    
    with tqdm(total=num_batches, desc="배치 작업 준비", unit="batch") as pbar:
        for i, current_batch_size in enumerate(batch_sizes):
            seed_offset = i * 1000 + random.randint(1, 999)  # 더 다양한 시드
            id_task = generate_unique_ids_batch_parallel(current_batch_size, seed_offset)
            account_task = generate_unique_accounts_batch_parallel(current_batch_size, seed_offset + 500)
            
            id_tasks.append(id_task)
            account_tasks.append(account_task)
            pbar.update(1)
    
    # ID와 계정 생성 병렬 실행
    print("🔄 ID 및 계정 병렬 처리 중...")
    start_time = time.time()
    
    with tqdm(total=2, desc="ID/계정 생성", unit="단계") as pbar:
        pbar.set_description("ID 생성 중")
        all_id_batches = dask.compute(*id_tasks)
        pbar.update(1)
        
        pbar.set_description("계정 생성 중")
        all_account_batches = dask.compute(*account_tasks)
        pbar.update(1)
    
    id_time = time.time() - start_time
    print(f"✅ ID 및 계정 생성 완료 ({id_time:.2f}초)")
    
    # 데이터 수집 및 정확한 개수 맞추기
    print("🔧 데이터 수집 및 정리 중...")
    all_ids = []
    all_accounts = []
    
    with tqdm(total=len(all_id_batches), desc="ID 수집", unit="batch") as pbar:
        for ids_batch in all_id_batches:
            all_ids.extend(ids_batch)
            pbar.update(1)
    
    with tqdm(total=len(all_account_batches), desc="계정 수집", unit="batch") as pbar:
        for accounts_batch in all_account_batches:
            all_accounts.extend(accounts_batch)
            pbar.update(1)
    
    print(f"📊 생성된 데이터: ID {len(all_ids):,}개, 계정 {len(all_accounts):,}개")
    
    # 중복 제거 후 정확한 개수만 선택
    with tqdm(total=3, desc="데이터 정리", unit="단계") as pbar:
        pbar.set_description("ID 중복 제거")
        unique_ids = list(set(all_ids))
        pbar.update(1)
        
        pbar.set_description("계정 중복 제거")
        unique_accounts = list(set(all_accounts))
        pbar.update(1)
        
        # 정확한 개수만 선택
        pbar.set_description("개수 조정")
        if len(unique_ids) >= num_users and len(unique_accounts) >= num_users:
            final_ids = unique_ids[:num_users]
            final_accounts = unique_accounts[:num_users]
        else:
            print(f"⚠️  부족한 데이터 감지: ID {len(unique_ids):,}개, 계정 {len(unique_accounts):,}개")
            # 부족한 경우 추가 생성
            needed_ids = max(0, num_users - len(unique_ids))
            needed_accounts = max(0, num_users - len(unique_accounts))
            
            if needed_ids > 0:
                print(f"🔄 부족한 ID {needed_ids:,}개 추가 생성...")
                additional_ids = [str(uuid.uuid4()) for _ in range(needed_ids)]
                unique_ids.extend(additional_ids)
            
            if needed_accounts > 0:
                print(f"🔄 부족한 계정 {needed_accounts:,}개 추가 생성...")
                fake_extra = Faker('ko_KR')
                additional_accounts = []
                for i in range(needed_accounts):
                    account = f"user_extra_{i}_{random.randint(10000, 99999)}"
                    additional_accounts.append(account)
                unique_accounts.extend(additional_accounts)
            
            final_ids = unique_ids[:num_users]
            final_accounts = unique_accounts[:num_users]
        
        pbar.update(1)
    
    print(f"📈 최종 데이터: ID {len(final_ids):,}개, 계정 {len(final_accounts):,}개")
    
    # 2단계: 정확한 개수의 사용자 데이터 생성
    print(f"\n2️⃣ 정확히 {num_users:,}명의 사용자 데이터 생성...")
    
    # 새로운 배치 크기 계산 (정확한 개수 기준)
    actual_num_batches = (num_users + batch_size - 1) // batch_size
    actual_batch_sizes = [batch_size] * (actual_num_batches - 1) + [num_users - (actual_num_batches - 1) * batch_size]
    
    user_tasks = []
    
    with tqdm(total=actual_num_batches, desc="사용자 배치 준비", unit="batch") as pbar:
        start_idx = 0
        for i, current_batch_size in enumerate(actual_batch_sizes):
            end_idx = start_idx + current_batch_size
            
            ids_batch = final_ids[start_idx:end_idx]
            accounts_batch = final_accounts[start_idx:end_idx]
            
            seed_offset = i * 2000 + random.randint(1, 1999)
            task = create_user_batch_with_locale(ids_batch, accounts_batch, seed_offset, locale)
            user_tasks.append(task)
            start_idx = end_idx
            
            pbar.update(1)
    
    # 사용자 데이터 생성 병렬 실행
    print("🔄 사용자 데이터 병렬 처리 중...")
    user_start_time = time.time()
    
    with tqdm(total=num_users, desc="사용자 데이터 생성", unit="users") as pbar:
        results = dask.compute(*user_tasks)
        
        completed_users = 0
        for batch_result in results:
            completed_users += len(batch_result)
            pbar.update(len(batch_result))
    
    user_time = time.time() - user_start_time
    print(f"✅ 사용자 데이터 생성 완료 ({user_time:.2f}초)")
    
    # 결과 병합
    print("📦 결과 병합 중...")
    user_seed = []
    with tqdm(total=len(results), desc="배치 병합", unit="batch") as pbar:
        for batch_result in results:
            user_seed.extend(batch_result)
            pbar.update(1)
    
    # 최종 검증
    if len(user_seed) != num_users:
        print(f"⚠️  개수 불일치: 예상 {num_users:,}개, 실제 {len(user_seed):,}개")
        user_seed = user_seed[:num_users]  # 정확한 개수로 자르기
    
    print(f"✅ 최종 생성된 사용자 수: {len(user_seed):,}명")
    return user_seed

# 실행
user_seed = generate_users_with_dask_optimized(NUM_USERS, batch_size=5000, locale='ko_KR')

🚀 최적화된 Dask로 정확히 50,000명의 한국 사용자 데이터 생성 시작...
📊 여유분 포함 55,000개 생성 목표 (실제 필요: 50,000개)
📊 총 11개 배치로 병렬 처리 (배치 크기: 5,000)

1️⃣ ID 및 계정 병렬 생성...


배치 작업 준비: 100%|██████████| 11/11 [00:00<00:00, 12801.70batch/s]


🔄 ID 및 계정 병렬 처리 중...


계정 생성 중: 100%|██████████| 2/2 [00:00<00:00,  2.21단계/s]


✅ ID 및 계정 생성 완료 (0.91초)
🔧 데이터 수집 및 정리 중...


...

🔄 사용자 데이터 병렬 처리 중...


사용자 데이터 생성: 100%|██████████| 50000/50000 [05:23<00:00, 154.78users/s]


✅ 사용자 데이터 생성 완료 (323.05초)
📦 결과 병합 중...


배치 병합: 100%|██████████| 10/10 [00:00<00:00, 11628.23batch/s]

✅ 최종 생성된 사용자 수: 50,000명


In [25]:
user_seed

[{'id': '4218a01b-f829-4c6d-adff-3b5d89d214cd',
  'account': 'junhyeog83_1782',
  'password': 'pbkdf2_sha256$260000$HfKUxXtE8eFUCq6GjQNfqw==$vkgfa/DjgvgxWyQsQSUtvc1Eqi/ba7352IIx4J7S8Qg=',
  'age': 25,
  'name': '이영일',
  'gender': 'M',
  'created_at': '2024-12-18T08:16:23.499563Z',
  'updated_at': '2025-01-13T08:16:23.499563Z',
  'address': '충청남도 원주시 가락길 557-74'},
 {'id': '4d3de5f6-4782-473b-89c5-08e75617a0c3',
  'account': 'coesumin_3508',
  'password': 'pbkdf2_sha256$260000$8t7jQGkfWyrjeqBd02v5Xg==$Pm20x0gon2zp6MZWoWn0b/0p4xa0gq3UBK+8dTmm8GI=',
  'age': 37,
  'name': '강윤서',
  'gender': 'M',
  'created_at': '2025-03-30T15:39:35.623579Z',
  'updated_at': '2025-03-30T15:39:35.623579Z',
  'address': '서울특별시 서대문구 서초중앙7가 479-56'},
 {'id': 'c90a93dc-e64c-4821-b774-48652d49742e',
  'account': 'cayeeun_8797',
  'password': 'pbkdf2_sha256$260000$0WLquI6fkM0T4/YHA9OTXw==$faQPuuWIRgNXytZRG/PucTdJfX8lKJpC/+CVoxq+F3Q=',
  'age': 29,
  'name': '박은경',
  'gender': 'M',
  'created_at': '2025-01-23T08:34

In [26]:
len(user_seed)

50000

In [27]:
user_seed[:1]

[{'id': '4218a01b-f829-4c6d-adff-3b5d89d214cd',
  'account': 'junhyeog83_1782',
  'password': 'pbkdf2_sha256$260000$HfKUxXtE8eFUCq6GjQNfqw==$vkgfa/DjgvgxWyQsQSUtvc1Eqi/ba7352IIx4J7S8Qg=',
  'age': 25,
  'name': '이영일',
  'gender': 'M',
  'created_at': '2024-12-18T08:16:23.499563Z',
  'updated_at': '2025-01-13T08:16:23.499563Z',
  'address': '충청남도 원주시 가락길 557-74'}]

In [28]:
# User 관련 정보를 쉽게 찾을 수 있도록 딕셔너리 생성
user_info = {}
for user in user_seed:
    user_info[user['id']] = user['created_at']

In [29]:
len(user_info)

50000

In [30]:
# 3) product 테이블 (Parent)
product_ids = product_iso["id"].values  # 데이터프레임에 존재하는 id만 사용

In [31]:
# 4) brand 테이블 (Parent)
brand_ids = product_iso["id"].values

In [62]:
# # 4) promotion 테이블 (Independent)
# promotion_seed = []
# for i in range(1, NUM_PROMOTIONS+1):
#     start = fake.date_time_between('-1y','-6m')
#     promotion_seed.append({
#         'id':         i,
#         'img_url':    fake.image_url(),
#         'start_time': start,
#         'end_time':   start + timedelta(days=random.randint(7,60))
#     })

In [63]:
# promotion_seed[:1]

## 3. 참조 데이터 생성
- cart_seed
- order_seed

### 3-1. Cart

In [32]:
@delayed
def generate_cart_batch(
    user_batch,
    product_seed,
    user_created_map,
    price_col="discounted_price",
    multi_item_rate=0.7,
    max_items_per_cart=5,
    multi_quantity_rate=0.1,
    max_qty=3,
    min_after_user=timedelta(hours=1),
    max_after_user=timedelta(days=30),
    seed_offset=0
):
    """장바구니 배치 생성 (기존 로직 유지)"""
    fake_local = Faker('ko_KR')
    fake_local.seed_instance(seed_offset)
    rng = np.random.default_rng(seed_offset)
    
    # 상품 데이터 준비
    if hasattr(product_seed, "to_dict"):
        prod_list = product_seed.to_dict('records')
    else:
        prod_list = product_seed
    product_map = {p['id']: p for p in prod_list}
    product_ids = list(product_map.keys())
    
    batch_carts = []
    
    for cid, uid in enumerate(user_batch, start=1):
        u_ct = user_created_map[uid]
        start = u_ct + min_after_user
        end = u_ct + max_after_user
        cart_ct = fake_local.date_time_between(start_date=start, end_date=end)
        
        updated_at = cart_ct + timedelta(days=int(rng.integers(0, 8)))
        
        # 다중 상품 여부 결정
        if rng.random() < multi_item_rate:
            n_items = int(rng.integers(2, max_items_per_cart + 1))
        else:
            n_items = 1
        
        # 상품 선택
        pids = rng.choice(product_ids, size=n_items, replace=False).tolist()
        
        # 수량 결정
        qtys = [
            int(rng.integers(2, max_qty + 1)) if rng.random() < multi_quantity_rate else 1
            for _ in range(n_items)
        ]
        
        # products 리스트에 price 필드 추가
        product_list = [
            {
                'product_id': pid,
                'quantity': qty,
                f'{price_col}': product_map[pid].get(f'{price_col}')
            }
            for pid, qty in zip(pids, qtys)
        ]
        
        batch_carts.append({
            'id': cid,
            'user_id': uid,
            'cart_items': product_list,
            'created_at': convert_to_iso8601_with_utc(cart_ct),
            'updated_at': convert_to_iso8601_with_utc(updated_at),
        })
    
    return batch_carts

def generate_cart_with_rates_dask(
    total_count: int,
    user_seed: list,
    product_seed,
    price_col="discounted_price",
    multi_item_rate: float = 0.2,
    max_items_per_cart: int = 5,
    multi_quantity_rate: float = 0.1,
    max_qty: int = 5,
    min_after_user: timedelta = timedelta(hours=0),
    max_after_user: timedelta = timedelta(days=7),
    batch_size: int = 2000
):
    """Dask를 사용한 병렬 장바구니 데이터 생성 (기존 로직 유지)"""
    print(f"🛒 Dask로 장바구니 데이터 생성 시작... (총 {total_count:,}개)")
    
    # 0) user_id → created_at lookup (ISO8601 문자열을 datetime으로 변환)
    user_created_map = {}
    for u in user_seed:
        if isinstance(u['created_at'], str):
            user_created_map[str(u['id'])] = pd.to_datetime(u['created_at']).to_pydatetime()
        else:
            user_created_map[str(u['id'])] = u['created_at']
    
    user_ids = list(user_created_map.keys())
    
    # 총 카트 수가 사용 가능한 유저 수보다 많으면 에러
    if total_count > len(user_ids):
        raise ValueError(f"total_count={total_count} exceeds unique user count={len(user_ids)}")
    
    # user_id를 중복 없이 샘플링
    rng = np.random.default_rng(42)
    user_choices = rng.choice(user_ids, size=total_count, replace=False).tolist()
    
    # 배치로 분할
    user_batches = [user_choices[i:i + batch_size] for i in range(0, len(user_choices), batch_size)]
    num_batches = len(user_batches)
    
    print(f"📊 총 {num_batches}개 배치로 병렬 처리 (배치 크기: {batch_size:,})")
    
    # 배치별 작업 생성
    cart_tasks = []
    with tqdm(total=num_batches, desc="장바구니 배치 준비", unit="batch") as pbar:
        for i, user_batch in enumerate(user_batches):
            seed_offset = i * 1000 + random.randint(1, 999)
            
            task = generate_cart_batch(
                user_batch=user_batch,
                product_seed=product_seed,
                user_created_map=user_created_map,
                price_col=price_col,
                multi_item_rate=multi_item_rate,
                max_items_per_cart=max_items_per_cart,
                multi_quantity_rate=multi_quantity_rate,
                max_qty=max_qty,
                min_after_user=min_after_user,
                max_after_user=max_after_user,
                seed_offset=seed_offset
            )
            cart_tasks.append(task)
            pbar.update(1)
    
    # 병렬 실행
    print("🔄 장바구니 병렬 처리 중...")
    start_time = time.time()
    
    with tqdm(desc="장바구니 생성", unit="batch") as pbar:
        results = dask.compute(*cart_tasks)
        pbar.update(len(results))
    
    # 결과 병합 및 ID 재조정
    cart_seed = []
    current_id = 1
    
    with tqdm(total=len(results), desc="장바구니 데이터 병합", unit="batch") as pbar:
        for batch_result in results:
            for cart in batch_result:
                cart['id'] = current_id
                current_id += 1
                cart_seed.append(cart)
            pbar.update(1)
    
    cart_time = time.time() - start_time
    print(f"✅ 장바구니 생성 완료 ({cart_time:.2f}초, 총 {len(cart_seed):,}개)")
    
    return cart_seed


In [35]:
total_start_time = time.time()

# Cart 데이터 생성
cart_seed = generate_cart_with_rates_dask(
    total_count=NUM_CARTS,
    user_seed=user_seed,
    product_seed=product,
    price_col="discounted_price",
    multi_item_rate=0.7,
    max_items_per_cart=5,
    multi_quantity_rate=0.1,
    max_qty=3,
    min_after_user=timedelta(hours=1),
    max_after_user=timedelta(days=30),
    batch_size=3000
)

total_time = time.time() - total_start_time
print("\n" + "=" * 80)
print("🎉 전체 데이터 생성 완료!")
print("=" * 80)
print(f"⏱️  총 실행 시간: {total_time:.2f}초")
print(f"🛒 장바구니: {len(cart_seed):,}개")
print(f"⚡ 처리 속도: {(len(cart_seed))/total_time:.0f}개/초")

🛒 Dask로 장바구니 데이터 생성 시작... (총 50,000개)
📊 총 17개 배치로 병렬 처리 (배치 크기: 3,000)


장바구니 배치 준비: 100%|██████████| 17/17 [00:03<00:00,  5.05batch/s]


🔄 장바구니 병렬 처리 중...


장바구니 생성: 17batch [00:16,  1.02batch/s]
장바구니 데이터 병합: 100%|██████████| 17/17 [00:00<00:00, 3514.72batch/s]

✅ 장바구니 생성 완료 (16.74초, 총 50,000개)

🎉 전체 데이터 생성 완료!
⏱️  총 실행 시간: 27.33초
🛒 장바구니: 50,000개
⚡ 처리 속도: 1830개/초


In [ ]:
# def generate_cart_with_rates(
#     total_count: int,
#     user_seed: list,
#     product_seed,
#     price_col = "discounted_price",
#     multi_item_rate: float = 0.2,
#     max_items_per_cart: int = 5,
#     multi_quantity_rate: float = 0.1,
#     max_qty: int = 5,
#     min_after_user: timedelta = timedelta(hours=0),
#     max_after_user: timedelta = timedelta(days=7),
#     fake=None,
#     rng=None
# ) -> list:
#     """
#     Purpose:
#         - 사용자 생성일(user_seed) 이후 cart.created_at 생성
#         - 상품(product_seed) 참조하여 cart당 다중상품·다중수량 비율 제어
#         - user_id는 전부 유일하게(중복 방지)
#         - product 내 price 필드 포함

#     Returns:
#         - List[dict] 각 dict:
#             {
#                 'id': int,
#                 'user_id': str,
#                 'products': [
#                     {
#                         'product_id': int,
#                         'quantity': int,
#                         'price': float
#                     }, …
#                 ],
#                 'created_at': str (ISO8601),
#                 'updated_at': str (ISO8601)
#             }
#     """
#     if fake is None:
#         fake = Faker('ko_KR')
#     if rng is None:
#         rng = np.random.default_rng(42)

#     # 0) user_id → created_at lookup (ISO8601 문자열을 datetime으로 변환)
#     user_created_map = {}
#     for u in user_seed:
#         # ISO8601 문자열을 datetime으로 변환
#         if isinstance(u['created_at'], str):
#             user_created_map[str(u['id'])] = pd.to_datetime(u['created_at']).to_pydatetime()
#         else:
#             user_created_map[str(u['id'])] = u['created_at']
    
#     user_ids = list(user_created_map.keys())

#     # 총 카트 수가 사용 가능한 유저 수보다 많으면 에러
#     if total_count > len(user_ids):
#         raise ValueError(f"total_count={total_count} exceeds unique user count={len(user_ids)}")

#     # 1) 상품 리스트 및 매핑 준비
#     if hasattr(product_seed, "to_dict"):
#         prod_list = product_seed.to_dict('records')
#     else:
#         prod_list = product_seed
#     product_map = {p['id']: p for p in prod_list}
#     product_ids = list(product_map.keys())

#     # 2) user_id를 중복 없이 샘플링
#     user_choices = rng.choice(user_ids, size=total_count, replace=False).tolist()

#     carts = []
#     for cid, uid in enumerate(user_choices, start=1):
#         u_ct = user_created_map[uid]
#         start = u_ct + min_after_user
#         end = u_ct + max_after_user
#         cart_ct = fake.date_time_between(start_date=start, end_date=end)

#         updated_at = cart_ct + timedelta(days=int(rng.integers(0, 8)))

#         if rng.random() < multi_item_rate:
#             n_items = int(rng.integers(2, max_items_per_cart + 1))
#         else:
#             n_items = 1

#         pids = rng.choice(product_ids, size=n_items, replace=False).tolist()

#         qtys = [
#             int(rng.integers(2, max_qty + 1)) if rng.random() < multi_quantity_rate else 1
#             for _ in range(n_items)
#         ]

#         # products 리스트에 price 필드 추가
#         product_list = [
#             {
#                 'product_id': pid,
#                 'quantity': qty,
#                 f'{price_col}': product_map[pid].get(f'{price_col}')
#             }
#             for pid, qty in zip(pids, qtys)
#         ]

#         carts.append({
#             'id': cid,
#             'user_id': uid,
#             'cart_items': product_list,
#             'created_at': convert_to_iso8601_with_utc(cart_ct),    # ISO8601 변환
#             'updated_at': convert_to_iso8601_with_utc(updated_at), # ISO8601 변환
#         })

#     return carts

In [37]:
# # 예시: 이미 아래 변수가 정의되어 있다고 가정
# # NUM_CARTS      = 5000
# # user_seed      = [...]            # List[dict], 각 dict에 'id' 및 'created_at' 포함
# # product        = pd.DataFrame(...) # 또는 List[dict], 'id' 및 'discounted_price' 컬럼 포함
# # fake           = Faker('ko_KR')
# # rng            = np.random.default_rng(42)

# cart_seed = generate_cart_with_rates(
#     total_count=NUM_CARTS,
#     user_seed=user_seed,
#     product_seed=product,        # DataFrame 또는 List[dict]
#     price_col="discounted_price",
#     multi_item_rate=0.7,         # 70% 카트에 2개 이상 상품
#     max_items_per_cart=5,        # 카트당 최대 5개 상품
#     multi_quantity_rate=0.1,     # 10% 상품은 quantity>1
#     max_qty=3,                   # 상품당 최대 수량 3
#     min_after_user=timedelta(hours=1),   # user 생성 1시간 후부터
#     max_after_user=timedelta(days=30),   # 최대 30일 후까지
#     fake=fake,
#     # rng=rng
# )

# # 결과 확인
# print(cart_seed[0])

In [38]:
cart_seed[1]

{'id': 2,
 'user_id': 'c97e56ca-6ff9-48a8-96b3-ca9d6fb4136b',
 'cart_items': [{'product_id': 4157330,
   'quantity': 1,
   'discounted_price': 122550},
  {'product_id': 3551090, 'quantity': 1, 'discounted_price': 39900},
  {'product_id': 4494116, 'quantity': 1, 'discounted_price': 115500}],
 'created_at': '2025-02-14T08:17:18.207795Z',
 'updated_at': '2025-02-15T08:17:18.207795Z'}

In [39]:
# 1) 다중 상품(cart) 검출
multi_item_carts = [c for c in cart_seed if len(c['cart_items']) > 1]

# 2) 전체 cart 대비 다중 상품 카트 비율 및 개수 출력
total = len(cart_seed)
multi_count = len(multi_item_carts)
print(f"전체 Cart 수: {total}")
print(f"다중 상품 Cart 수: {multi_count} ({multi_count/total:.2%})")

# 3) 예시 샘플 5건 확인
for c in multi_item_carts[:5]:
    print(f"id={c['id']}, user_id={c['user_id']}, product 개수={len(c['cart_items'])}")
    print(c['cart_items'])

전체 Cart 수: 50000
다중 상품 Cart 수: 34954 (69.91%)
id=1, user_id=a2ed8ab6-4f78-4534-9214-95a189223c49, product 개수=4
[{'product_id': 4538543, 'quantity': 1, 'discounted_price': 630400}, {'product_id': 4352077, 'quantity': 1, 'discounted_price': 195200}, {'product_id': 4520638, 'quantity': 1, 'discounted_price': 15900}, {'product_id': 4535634, 'quantity': 1, 'discounted_price': 199000}]
id=2, user_id=c97e56ca-6ff9-48a8-96b3-ca9d6fb4136b, product 개수=3
[{'product_id': 4157330, 'quantity': 1, 'discounted_price': 122550}, {'product_id': 3551090, 'quantity': 1, 'discounted_price': 39900}, {'product_id': 4494116, 'quantity': 1, 'discounted_price': 115500}]
id=3, user_id=46a13982-a59e-46ef-a0f3-ec6809a10644, product 개수=2
[{'product_id': 4851401, 'quantity': 1, 'discounted_price': 125000}, {'product_id': 4145221, 'quantity': 1, 'discounted_price': 119900}]
id=4, user_id=b4e0a18c-285a-4104-88aa-08e5a39abd4e, product 개수=5
[{'product_id': 4427484, 'quantity': 1, 'discounted_price': 72000}, {'product_id'

### 3-2. Order

In [33]:
@delayed
def generate_orders_batch(
    batch_info,
    all_cart_seed,
    user_seed,
    product_seed,
    price_col="discounted_price",
    seed_offset=0
):
    """주문 배치 생성 (기존 로직 유지)"""
    fake_local = Faker('ko_KR')
    fake_local.seed_instance(seed_offset)
    rng = np.random.default_rng(seed_offset)
    
    scenario1_carts, scenario2_indices, start_order_id = batch_info
    
    user_ids = [str(u['id']) for u in user_seed]
    
    if hasattr(product_seed, "to_dict"):
        prods = product_seed.to_dict('records')
    else:
        prods = product_seed
    price_map = {p['id']: p[f'{price_col}'] for p in prods}
    
    # user_seed의 created_at을 datetime으로 변환해서 매핑
    user_created_map = {}
    for u in user_seed:
        if isinstance(u['created_at'], str):
            user_created_map[str(u['id'])] = pd.to_datetime(u['created_at']).to_pydatetime()
        else:
            user_created_map[str(u['id'])] = u['created_at']
    
    # cart_seed의 datetime도 변환
    def parse_cart_datetime(cart_dt):
        if isinstance(cart_dt, str):
            return pd.to_datetime(cart_dt).to_pydatetime()
        return cart_dt
    
    # TID 생성 함수
    def generate_tid():
        chars = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        return 'T' + ''.join(rng.choice(list(chars), size=19))
    
    # Status와 Payment status 관계를 고려한 생성 함수
    def generate_status_and_payment():
        status_weights = {
            'pending_paid': 0.10,
            'paid': 0.40,
            'shipping': 0.05,
            'shipped': 0.25,
            'cancelled': 0.20
        }
        
        statuses = list(status_weights.keys())
        weights = list(status_weights.values())
        status = str(rng.choice(statuses, p=weights))
        
        if status == 'pending_paid':
            payment_options = [
                "pending_payment",
                "pending_kakao_ready", 
                "ready_kakao",
                "failed_kakao_ready",
                "failed_kakao_ready_response",
                "failed_kakao_approve"
            ]
            payment_weights = [0.3, 0.2, 0.2, 0.1, 0.1, 0.1]
            payment_status = str(rng.choice(payment_options, p=payment_weights))
            
        elif status == 'paid':
            payment_status = "paid_kakao"
            
        elif status in ['shipping', 'shipped']:
            payment_status = "paid_kakao"
            
        elif status == 'cancelled':
            payment_options = ["cancelled_kakao", "failed_kakao_approve"]
            payment_weights = [0.8, 0.2]
            payment_status = str(rng.choice(payment_options, p=payment_weights))
        
        return status, payment_status
    
    orders = []
    next_id = start_order_id
    
    # 시나리오 1: Cart → Order 전환
    for cart in scenario1_carts:
        status, payment_status = generate_status_and_payment()
        
        o = {
            'id': next_id,
            'status': status,
            'tid': generate_tid(),
            'payment_status': payment_status,
            'is_from_cart': rng.random() < 0.5
        }
        next_id += 1
        
        others = [u for u in user_ids if u != str(cart['user_id'])]
        o['user_id'] = random.choice(others)
        
        cart_qtys = [item['quantity'] for item in cart['cart_items']]
        forbidden = [item['product_id'] for item in cart['cart_items']]
        pool = [pid for pid in price_map if pid not in forbidden]
        chosen = rng.choice(pool, size=len(cart_qtys), replace=False)
        
        items = []
        total = 0
        for pid, qty in zip(chosen, cart_qtys):
            pid = int(pid)
            price = price_map[pid]
            items.append({
                'product_id': pid,
                'quantity': qty,
                f'{price_col}': price
            })
            total += qty * price
        o['order_items'] = items
        o['total_price'] = total
        
        cart_created_at = parse_cart_datetime(cart['created_at'])
        order_created_at = fake_local.date_time_between(
            start_date=cart_created_at,
            end_date='now'
        )
        delta_u = int(rng.integers(0, 7*24*3600))
        order_updated_at = order_created_at + timedelta(seconds=delta_u)
        
        o['created_at'] = convert_to_iso8601_with_utc(order_created_at)
        o['updated_at'] = convert_to_iso8601_with_utc(order_updated_at)
        
        orders.append(o)
    
    # 시나리오 2: 기타 경우
    for idx in scenario2_indices:
        cart = all_cart_seed[idx]
        
        if rng.random() < 0.5:
            uid = str(cart['user_id'])
        else:
            others = [u for u in user_ids if u != str(cart['user_id'])]
            uid = random.choice(others)
        
        status, payment_status = generate_status_and_payment()
        
        o = {
            'id': next_id,
            'status': status,
            'tid': generate_tid(),
            'payment_status': payment_status,
            'user_id': uid,
            'is_from_cart': rng.random() < 0.5
        }
        next_id += 1
        
        items = []
        total = 0
        for item in cart['cart_items']:
            pid = int(item['product_id'])
            qty = item['quantity']
            price = price_map[pid]
            items.append({'product_id': pid, 'quantity': qty, f'{price_col}': price})
            total += qty * price
        o['order_items'] = items
        o['total_price'] = total
        
        cart_created_at = parse_cart_datetime(cart['created_at'])
        
        if rng.random() < 0.7:
            # 2-1: order.created_at < cart.created_at
            user_ct = user_created_map[uid]
            order_created_at = fake_local.date_time_between(
                start_date=user_ct,
                end_date=cart_created_at
            )
        else:
            # 2-2: order.created_at > cart.created_at
            order_created_at = fake_local.date_time_between(
                start_date=cart_created_at,
                end_date='now'
            )
        
        delta_u = int(rng.integers(0, 7*24*3600))
        order_updated_at = order_created_at + timedelta(seconds=delta_u)
        
        o['created_at'] = convert_to_iso8601_with_utc(order_created_at)
        o['updated_at'] = convert_to_iso8601_with_utc(order_updated_at)
        
        orders.append(o)
    
    return orders

def generate_orders_dask(
    cart_seed: list,
    user_seed: list,
    product_seed,
    NUM_ORDERS: int,
    price_col="discounted_price",
    scenario1_ratio: float = 0.8,
    batch_size: int = 2000
):
    """Dask를 사용한 병렬 주문 데이터 생성 (기존 로직 유지)"""
    print(f"📦 Dask로 주문 데이터 생성 시작... (총 {NUM_ORDERS:,}개)")
    
    rng = np.random.default_rng(42)
    
    n1 = int(NUM_ORDERS * scenario1_ratio)
    n2 = NUM_ORDERS - n1
    
    # 시나리오 1용 카트 샘플링
    carts1 = random.sample(cart_seed, k=n1)
    
    # 시나리오 2용 인덱스 생성
    idxs2 = rng.integers(0, len(cart_seed), size=n2)
    
    # 배치로 분할
    scenario1_batches = [carts1[i:i + batch_size] for i in range(0, len(carts1), batch_size)]
    scenario2_batches = [idxs2[i:i + batch_size] for i in range(0, len(idxs2), batch_size)]
    
    # 배치 정보 생성
    batch_infos = []
    order_id_counter = 1
    
    # 시나리오1 배치들
    for batch in scenario1_batches:
        batch_infos.append((batch, [], order_id_counter))
        order_id_counter += len(batch)
    
    # 시나리오2 배치들
    for batch in scenario2_batches:
        batch_infos.append(([], batch.tolist(), order_id_counter))
        order_id_counter += len(batch)
    
    print(f"📊 총 {len(batch_infos)}개 배치로 병렬 처리")
    
    # 배치별 작업 생성
    order_tasks = []
    with tqdm(total=len(batch_infos), desc="주문 배치 준비", unit="batch") as pbar:
        for i, batch_info in enumerate(batch_infos):
            seed_offset = i * 3000 + random.randint(1, 2999)
            
            task = generate_orders_batch(
                batch_info=batch_info,
                all_cart_seed=cart_seed,
                user_seed=user_seed,
                product_seed=product_seed,
                price_col=price_col,
                seed_offset=seed_offset
            )
            order_tasks.append(task)
            pbar.update(1)
    
    # 병렬 실행
    print("🔄 주문 병렬 처리 중...")
    start_time = time.time()
    
    with tqdm(desc="주문 생성", unit="batch") as pbar:
        results = dask.compute(*order_tasks)
        pbar.update(len(results))
    
    # 결과 병합
    order_seed = []
    with tqdm(total=len(results), desc="주문 데이터 병합", unit="batch") as pbar:
        for batch_result in results:
            order_seed.extend(batch_result)
            pbar.update(1)
    
    order_time = time.time() - start_time
    print(f"✅ 주문 생성 완료 ({order_time:.2f}초, 총 {len(order_seed):,}개)")
    
    return order_seed

In [47]:
total_start_time = time.time()

# Order 데이터 생성
order_seed = generate_orders_dask(
    cart_seed=cart_seed,
    user_seed=user_seed,
    product_seed=product,
    NUM_ORDERS=50000,
    price_col="discounted_price",
    scenario1_ratio=0.75,
    batch_size=2000
)

print("\n" + "=" * 80)
print("🎉 전체 데이터 생성 완료!")
print("=" * 80)
print(f"⏱️  총 실행 시간: {total_time:.2f}초")
print(f"📦 주문: {len(order_seed):,}개")
print(f"⚡ 처리 속도: {(len(order_seed))/total_time:.0f}개/초")

📦 Dask로 주문 데이터 생성 시작... (총 50,000개)
📊 총 26개 배치로 병렬 처리


주문 배치 준비: 100%|██████████| 26/26 [01:32<00:00,  3.55s/batch]


🔄 주문 병렬 처리 중...


주문 생성: 26batch [05:05, 11.75s/batch]
주문 데이터 병합: 100%|██████████| 26/26 [00:00<00:00, 30166.50batch/s]

✅ 주문 생성 완료 (305.50초, 총 50,000개)

🎉 전체 데이터 생성 완료!
⏱️  총 실행 시간: 27.33초
📦 주문: 50,000개
⚡ 처리 속도: 1830개/초


In [ ]:
# def generate_orders(
#     cart_seed: list,
#     user_seed: list,
#     product_seed,
#     NUM_ORDERS: int,
#     price_col = "discounted_price",
#     scenario1_ratio: float = 0.8,
#     rng: np.random.Generator = None,
#     fake=None
# ) -> list:
#     """
#     주문 데이터 생성 함수

#     Args:
#       - cart_seed (List[dict]): Cart 데이터 리스트
#       - user_seed (List[dict]): User 시드 리스트 ({'id','created_at'} 포함)
#       - product_seed (DataFrame or List[dict]): 상품 시드 ('id','discounted_price' 포함)
#       - NUM_ORDERS (int): 총 생성할 주문 수
#       - scenario1_ratio (float): 시나리오① 비율 (0~1)
#       - rng (np.random.Generator, optional): 난수 발생기
#       - fake (Faker, optional): Faker 인스턴스 (created_at 생성용)

#     Returns:
#       - List[dict]: 생성된 주문 리스트 (created_at, updated_at은 ISO8601 형태)
#     """
#     if rng is None:
#         rng = np.random.default_rng(42)
#     if fake is None:
#         from faker import Faker
#         fake = Faker('ko_KR')

#     user_ids = [str(u['id']) for u in user_seed]

#     if hasattr(product_seed, "to_dict"):
#         prods = product_seed.to_dict('records')
#     else:
#         prods = product_seed
#     price_map = {p['id']: p[f'{price_col}'] for p in prods}

#     # user_seed의 created_at을 datetime으로 변환해서 매핑
#     user_created_map = {}
#     for u in user_seed:
#         if isinstance(u['created_at'], str):
#             user_created_map[str(u['id'])] = pd.to_datetime(u['created_at']).to_pydatetime()
#         else:
#             user_created_map[str(u['id'])] = u['created_at']

#     # cart_seed의 datetime도 변환
#     def parse_cart_datetime(cart_dt):
#         if isinstance(cart_dt, str):
#             return pd.to_datetime(cart_dt).to_pydatetime()
#         return cart_dt

#     # TID 생성 함수 (T + 19자리 숫자+문자)
#     def generate_tid():
#         """T로 시작하고 숫자+문자열로 20자리 랜덤값 생성"""
#         chars = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
#         return 'T' + ''.join(rng.choice(list(chars), size=19))

#     # Status와 Payment status 관계를 고려한 생성 함수
#     def generate_status_and_payment():
#         """
#         Status와 Payment status를 연관성 있게 생성
#         """
#         # 전체 주문 상태 분포 설정
#         status_weights = {
#             'pending_paid': 0.10,  # 결제 대기
#             'paid': 0.40,          # 결제 완료
#             'shipping': 0.05,      # 배송 중
#             'shipped': 0.25,       # 배송 완료
#             'cancelled': 0.20      # 취소
#         }
        
#         # 가중치를 기반으로 status 선택
#         statuses = list(status_weights.keys())
#         weights = list(status_weights.values())
#         status = str(rng.choice(statuses, p=weights))  # str()로 변환
        
#         # Status에 따른 Payment status 매핑
#         if status == 'pending_paid':
#             # 결제 대기 중인 경우
#             payment_options = [
#                 "pending_payment",
#                 "pending_kakao_ready", 
#                 "ready_kakao",
#                 "failed_kakao_ready",
#                 "failed_kakao_ready_response",
#                 "failed_kakao_approve"
#             ]
#             payment_weights = [0.3, 0.2, 0.2, 0.1, 0.1, 0.1]
#             payment_status = str(rng.choice(payment_options, p=payment_weights))  # str()로 변환
            
#         elif status == 'paid':
#             # 결제 완료된 경우
#             payment_status = "paid_kakao"  # 결제 완료는 무조건 paid_kakao
            
#         elif status in ['shipping', 'shipped']:
#             # 배송 중이거나 완료된 경우 (결제는 이미 완료)
#             payment_status = "paid_kakao"
            
#         elif status == 'cancelled':
#             # 취소된 경우
#             payment_options = ["cancelled_kakao", "failed_kakao_approve"]
#             payment_weights = [0.8, 0.2]  # 대부분 cancelled_kakao
#             payment_status = str(rng.choice(payment_options, p=payment_weights))  # str()로 변환
        
#         return status, payment_status

#     n1 = int(NUM_ORDERS * scenario1_ratio)
#     n2 = NUM_ORDERS - n1

#     orders = []
#     next_id = 1

#     # ── 시나리오 1: Cart → Order 전환 ─────────────────────────
#     carts1 = random.sample(cart_seed, k=n1)
#     for cart in carts1:
#         # Status와 Payment status를 연관성 있게 생성
#         status, payment_status = generate_status_and_payment()
        
#         o = {
#             'id': next_id,
#             'status': status,
#             'tid': generate_tid(),
#             'payment_status': payment_status,
#             'is_from_cart': rng.random() < 0.5
#         }
#         next_id += 1

#         others = [u for u in user_ids if u != str(cart['user_id'])]
#         o['user_id'] = random.choice(others)

#         cart_qtys = [item['quantity'] for item in cart['cart_items']]
#         forbidden = [item['product_id'] for item in cart['cart_items']]
#         pool = [pid for pid in price_map if pid not in forbidden]
#         chosen = rng.choice(pool, size=len(cart_qtys), replace=False)

#         items = []
#         total = 0
#         for pid, qty in zip(chosen, cart_qtys):
#             pid = int(pid)  # numpy.int64 → Python int 로 변환
#             price = price_map[pid]
#             items.append({
#                 'product_id': pid,
#                 'quantity': qty,
#                 f'{price_col}': price
#             })
#             total += qty * price
#         o['order_items'] = items
#         o['total_price']  = total

#         # cart datetime 파싱
#         cart_created_at = parse_cart_datetime(cart['created_at'])
        
#         order_created_at = fake.date_time_between(
#             start_date=cart_created_at,
#             end_date='now'
#         )
#         delta_u = int(rng.integers(0, 7*24*3600))
#         order_updated_at = order_created_at + timedelta(seconds=delta_u)

#         # ISO8601 변환
#         o['created_at'] = convert_to_iso8601_with_utc(order_created_at)
#         o['updated_at'] = convert_to_iso8601_with_utc(order_updated_at)

#         orders.append(o)

#     # ── 시나리오 2: 기타 경우 ─────────────────────────────────
#     idxs2 = rng.integers(0, len(cart_seed), size=n2)
#     for idx in idxs2:
#         cart = cart_seed[idx]

#         # 1) user_id 결정 (절반은 동일, 절반은 다른 유저)
#         if rng.random() < 0.5:
#             uid = str(cart['user_id'])
#         else:
#             others = [u for u in user_ids if u != str(cart['user_id'])]
#             uid    = random.choice(others)

#         # Status와 Payment status를 연관성 있게 생성
#         status, payment_status = generate_status_and_payment()

#         # 2) 기본 주문 필드 세팅
#         o = {
#             'id':             next_id,
#             'status':         status,
#             'tid':            generate_tid(),
#             'payment_status': payment_status,
#             'user_id':        uid,
#             'is_from_cart':   rng.random() < 0.5
#         }
#         next_id += 1

#         # 3) order_items, total_price 구성 (cart 그대로)
#         items = []
#         total = 0
#         for item in cart['cart_items']:
#             pid = int(item['product_id'])
#             qty = item['quantity']
#             price = price_map[pid]
#             items.append({'product_id': pid, 'quantity': qty, f'{price_col}': price})
#             total += qty * price
#         o['order_items'] = items
#         o['total_price']  = total

#         # cart datetime 파싱
#         cart_created_at = parse_cart_datetime(cart['created_at'])

#         # 4) 서브케이스 분기: 2-1 vs 2-2
#         #  - 2-1: 주문 먼저 → cart 다시 담기
#         #  - 2-2: cart 먼저 → 주문 (detail→order)
#         if rng.random() < 0.7:
#             # ── 2-1: order.created_at < cart.created_at ──
#             #   주문 시각은 user_created_at 이후, cart.created_at 이전
#             user_ct = user_created_map[uid]
#             order_created_at = fake.date_time_between(
#                 start_date=user_ct,
#                 end_date=cart_created_at
#             )
#         else:
#             # ── 2-2: order.created_at > cart.created_at ──
#             #   주문 시각은 cart.created_at 이후 아무 시점
#             order_created_at = fake.date_time_between(
#                 start_date=cart_created_at,
#                 end_date='now'
#             )

#         # 5) updated_at: created_at 이후 0~7일 랜덤
#         delta_u = int(rng.integers(0, 7*24*3600))
#         order_updated_at = order_created_at + timedelta(seconds=delta_u)

#         # ISO8601 변환
#         o['created_at'] = convert_to_iso8601_with_utc(order_created_at)
#         o['updated_at'] = convert_to_iso8601_with_utc(order_updated_at)

#         orders.append(o)
    
#     return orders

In [41]:
# order_seed = generate_orders(
#     cart_seed=cart_seed,
#     user_seed=user_seed,
#     product_seed=product,
#     NUM_ORDERS=10_000,
#     price_col="discounted_price",
#     scenario1_ratio=0.75,
#     # rng=rng
# )

In [42]:
order_seed[:3]

[{'id': 1,
  'status': 'pending_paid',
  'tid': 'TA3AXNAMAAD8K44UYXR7',
  'payment_status': 'ready_kakao',
  'is_from_cart': True,
  'user_id': '24fb5932-0593-49cd-ac0a-03611c0388ac',
  'order_items': [{'product_id': 2785964,
    'quantity': 1,
    'discounted_price': 258120},
   {'product_id': 2172957, 'quantity': 1, 'discounted_price': 245000},
   {'product_id': 4682254, 'quantity': 1, 'discounted_price': 39890}],
  'total_price': 543010,
  'created_at': '2025-06-11T07:25:39.474754Z',
  'updated_at': '2025-06-14T12:14:54.474754Z'},
 {'id': 2,
  'status': 'paid',
  'tid': 'T7JUQ0WI3USTBVAM0WJ9',
  'payment_status': 'paid_kakao',
  'is_from_cart': False,
  'user_id': 'ea335acf-0dd7-4ff1-827d-69bf7c3911b4',
  'order_items': [{'product_id': 3801512,
    'quantity': 1,
    'discounted_price': 96750},
   {'product_id': 3562272, 'quantity': 1, 'discounted_price': 118300},
   {'product_id': 4793514, 'quantity': 3, 'discounted_price': 79920}],
  'total_price': 454810,
  'created_at': '2024-11

In [43]:
# 1) 주문별 order_items 개수 확인
item_counts = [(o['id'], len(o['order_items'])) for o in order_seed]

# 2) 최대 품목 개수 찾기
max_count = max(count for _, count in item_counts)

# 3) 최대 품목을 가진 주문 필터링
max_orders = [o for o in order_seed if len(o['order_items']) == max_count]

print(f"가장 많은 order_items 개수: {max_count}")
# print("해당 주문들:")
# for o in max_orders:
#     print(f"order_id={o['id']}, user_id={o['user_id']}, order_items 개수={len(o['order_items'])}")
#     print(o['order_items'])

가장 많은 order_items 개수: 5


In [44]:
len(order_seed)

10000

## 4. 데이터 검증
- 참조한 데이터와 생성한 데이터셋이 동일한 값을 갖고 있는지 확인
    - 가격 일관성 (`price`, `quantity`)
    - id 일관성
- 시간 일관성 (`created_at`) 확인

### 4-1. Cart

In [45]:
cart_seed[:2]

[{'id': 1,
  'user_id': 'a2ed8ab6-4f78-4534-9214-95a189223c49',
  'cart_items': [{'product_id': 4538543,
    'quantity': 1,
    'discounted_price': 630400},
   {'product_id': 4352077, 'quantity': 1, 'discounted_price': 195200},
   {'product_id': 4520638, 'quantity': 1, 'discounted_price': 15900},
   {'product_id': 4535634, 'quantity': 1, 'discounted_price': 199000}],
  'created_at': '2025-05-08T17:54:25.226284Z',
  'updated_at': '2025-05-10T17:54:25.226284Z'},
 {'id': 2,
  'user_id': 'c97e56ca-6ff9-48a8-96b3-ca9d6fb4136b',
  'cart_items': [{'product_id': 4157330,
    'quantity': 1,
    'discounted_price': 122550},
   {'product_id': 3551090, 'quantity': 1, 'discounted_price': 39900},
   {'product_id': 4494116, 'quantity': 1, 'discounted_price': 115500}],
  'created_at': '2025-02-14T08:17:18.207795Z',
  'updated_at': '2025-02-15T08:17:18.207795Z'}]

In [46]:
# 1) user_id 중복 검사
user_ids = [str(c['user_id']) for c in cart_seed]
dupe_users = [uid for uid, cnt in Counter(user_ids).items() if cnt > 1]
if dupe_users:
    print(f"중복된 user_id 발견: {dupe_users}")
else:
    print("모든 user_id가 유일합니다.")

# 2) product_id 유효성 검사
#    product_ids_set 은 product DataFrame의 'id' 컬럼 참조
product_ids_set = set(product['id'].tolist())

invalid_product_carts = []
for c in cart_seed:
    for p in c['cart_items']:
        if p['product_id'] not in product_ids_set:
            invalid_product_carts.append(c['id'])
            break

if invalid_product_carts:
    print(f"잘못된 product_id를 가진 cart_id: {invalid_product_carts[:10]}{'...' if len(invalid_product_carts)>10 else ''}")
else:
    print("모든 product_id가 유효합니다.")

# 3) created_at ≤ updated_at 검사
invalid_time_carts = [c['id'] for c in cart_seed if c['created_at'] > c['updated_at']]
if invalid_time_carts:
    print(f"created_at > updated_at 인 cart_id: {invalid_time_carts[:10]}{'...' if len(invalid_time_carts)>10 else ''}")
else:
    print("모든 레코드에서 created_at ≤ updated_at 순서를 만족합니다.")

모든 user_id가 유일합니다.
모든 product_id가 유효합니다.
모든 레코드에서 created_at ≤ updated_at 순서를 만족합니다.


### 4-2. Order

In [48]:
@delayed
def validate_order_batch(
    order_batch,
    price_map,
    cart_by_user,
    cart_by_items,
    valid_statuses,
    seed_offset=0
):
    """주문 배치 검증 (병렬 처리용)"""
    
    # datetime 파싱 헬퍼 함수
    def parse_iso8601_to_datetime(dt_str):
        """ISO8601 문자열을 datetime 객체로 변환"""
        if isinstance(dt_str, str):
            return pd.to_datetime(dt_str).to_pydatetime()
        return dt_str
    
    batch_invalid = []
    
    for o in order_batch:
        errs = []
        uid = str(o['user_id'])
        o_ct = parse_iso8601_to_datetime(o['created_at'])

        # 2-a) status 유효성
        if o['status'] not in valid_statuses:
            errs.append(f"invalid status: {o['status']}")

        # 2-b) 가격·total_price 검증
        calc_total = 0
        for it in o['order_items']:
            pid, qty, pr = it['product_id'], it['quantity'], it['discounted_price']
            if pid not in price_map:
                errs.append(f"unknown product_id: {pid}")
            elif price_map[pid] != pr:
                errs.append(f"wrong price for {pid}: got {pr}, expect {price_map[pid]}")
            calc_total += qty * pr
        if o['total_price'] != calc_total:
            errs.append(f"total_price {o['total_price']} != computed {calc_total}")

        # 2-c) 시나리오① 검증 (최적화: 다른 사용자의 카트만 탐색)
        scenario1_ok = False
        o_pids = {itm['product_id'] for itm in o['order_items']}
        
        # 다른 사용자들의 카트만 확인
        for other_uid, other_carts in cart_by_user.items():
            if other_uid == uid:
                continue
            
            for c in other_carts:
                ct0 = parse_iso8601_to_datetime(c['created_at'])
                if not (ct0 <= o_ct <= ct0 + timedelta(days=7)):
                    continue
                
                cart_pids = {itm['product_id'] for itm in c['cart_items']}
                if cart_pids & o_pids:  # 교집합이 있으면 skip
                    continue
                if len(c['cart_items']) != len(o['order_items']):
                    continue
                
                scenario1_ok = True
                break
            
            if scenario1_ok:
                break

        # 2-d) 시나리오② 검증 (최적화: 동일한 상품조합만 탐색)
        scenario2_ok = False
        order_items_key = tuple(sorted((itm['product_id'], itm['quantity']) 
                                     for itm in o['order_items']))
        
        # 동일한 상품조합을 가진 카트들만 확인
        if order_items_key in cart_by_items:
            for c in cart_by_items[order_items_key]:
                ct0 = parse_iso8601_to_datetime(c['created_at'])
                # 시간 조건만 확인하면 됨 (상품조합은 이미 일치)
                if o_ct < ct0 or o_ct >= ct0:  # 2-1 또는 2-2 케이스
                    scenario2_ok = True
                    break

        # 둘 중 하나도 만족하지 않으면 invalid
        if not (scenario1_ok or scenario2_ok):
            errs.append("no matching scenario")

        if errs:
            batch_invalid.append({'order_id': o['id'], 'errors': errs})
    
    return batch_invalid

def validate_orders_with_dask(
    order_seed,
    cart_seed,
    product_seed,
    batch_size=1000
):
    """Dask를 사용한 병렬 주문 데이터 검증"""
    print(f"🔍 Dask로 주문 데이터 검증 시작... (총 {len(order_seed):,}개)")
    
    # 1) 참조용 데이터 준비 및 인덱스 생성
    print("📊 참조 데이터 준비 중...")
    
    if hasattr(product_seed, "to_dict"):
        prod_list = product_seed.to_dict('records')
    else:
        prod_list = product_seed
    price_map = {p['id']: p['discounted_price'] for p in prod_list}
    
    valid_statuses = {"pending_paid", "paid", "shipping", "shipped", "cancelled"}
    
    # 2) 카트 데이터 인덱싱
    print("🗂️  카트 데이터 인덱싱 중...")
    
    # 사용자별 카트 인덱스
    cart_by_user = {}
    with tqdm(total=len(cart_seed), desc="사용자별 인덱싱", unit="cart") as pbar:
        for c in cart_seed:
            user_id = str(c['user_id'])
            if user_id not in cart_by_user:
                cart_by_user[user_id] = []
            cart_by_user[user_id].append(c)
            pbar.update(1)
    
    # 상품-수량 조합별 카트 인덱스 (시나리오2용)
    cart_by_items = {}
    with tqdm(total=len(cart_seed), desc="상품조합별 인덱싱", unit="cart") as pbar:
        for c in cart_seed:
            items_key = tuple(sorted((item['product_id'], item['quantity']) 
                                   for item in c['cart_items']))
            if items_key not in cart_by_items:
                cart_by_items[items_key] = []
            cart_by_items[items_key].append(c)
            pbar.update(1)
    
    print(f"📈 인덱싱 완료: {len(cart_by_user)}명 사용자, {len(cart_by_items)}개 상품조합")
    
    # 3) 주문 데이터를 배치로 분할
    order_batches = [order_seed[i:i + batch_size] for i in range(0, len(order_seed), batch_size)]
    num_batches = len(order_batches)
    
    print(f"📊 총 {num_batches}개 배치로 병렬 검증 처리 (배치 크기: {batch_size:,})")
    
    # 4) 배치별 검증 작업 생성
    validation_tasks = []
    with tqdm(total=num_batches, desc="검증 배치 준비", unit="batch") as pbar:
        for i, order_batch in enumerate(order_batches):
            seed_offset = i * 1000 + i  # 각 배치별 고유 시드
            
            task = validate_order_batch(
                order_batch=order_batch,
                price_map=price_map,
                cart_by_user=cart_by_user,
                cart_by_items=cart_by_items,
                valid_statuses=valid_statuses,
                seed_offset=seed_offset
            )
            validation_tasks.append(task)
            pbar.update(1)
    
    # 5) 병렬 검증 실행
    print("🔄 주문 검증 병렬 처리 중...")
    start_time = time.time()
    
    with tqdm(total=len(order_seed), desc="주문 검증", unit="orders") as pbar:
        results = dask.compute(*validation_tasks)
        
        # 진행도 업데이트
        processed_orders = 0
        for batch_result in results:
            processed_orders += len(order_batches[results.index(batch_result)])
            pbar.update(len(order_batches[results.index(batch_result)]))
    
    # 6) 결과 병합
    print("📦 검증 결과 병합 중...")
    invalid_orders = []
    
    with tqdm(total=len(results), desc="결과 병합", unit="batch") as pbar:
        for batch_result in results:
            invalid_orders.extend(batch_result)
            pbar.update(1)
    
    validation_time = time.time() - start_time
    
    # 7) 결과 출력
    print(f"\n✅ 주문 검증 완료 ({validation_time:.2f}초)")
    print(f"📊 총 주문 건수: {len(order_seed):,}개")
    print(f"❌ 검증 실패 건수: {len(invalid_orders):,}개")
    print(f"✅ 검증 성공률: {((len(order_seed) - len(invalid_orders)) / len(order_seed) * 100):.2f}%")
    print(f"⚡ 검증 속도: {len(order_seed)/validation_time:.0f}개/초")
    
    if invalid_orders:
        print(f"\n❌ 검증 실패 예시 (상위 10건):")
        for i, rec in enumerate(invalid_orders[:10], 1):
            print(f"  {i:2d}. 주문 ID: {rec['order_id']}")
            for error in rec['errors']:
                print(f"      - {error}")
    else:
        print("\n🎉 모든 주문이 검증을 통과했습니다!")
    
    return invalid_orders

# =============================================================================
# 통계 분석 함수들 (선택사항)
# =============================================================================

def analyze_validation_results(invalid_orders):
    """검증 결과 상세 분석"""
    if not invalid_orders:
        print("🎉 분석할 오류가 없습니다!")
        return
    
    print("\n📈 검증 실패 원인 분석")
    print("=" * 50)
    
    # 오류 유형별 통계
    error_stats = {}
    for record in invalid_orders:
        for error in record['errors']:
            error_type = error.split(':')[0] if ':' in error else error
            error_stats[error_type] = error_stats.get(error_type, 0) + 1
    
    # 오류 유형별 출력
    print("오류 유형별 발생 횟수:")
    for error_type, count in sorted(error_stats.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(invalid_orders)) * 100
        print(f"  • {error_type}: {count:,}건 ({percentage:.1f}%)")
    
    # 가장 많은 오류를 가진 주문들
    error_counts = [(rec['order_id'], len(rec['errors'])) for rec in invalid_orders]
    error_counts.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n가장 많은 오류를 가진 주문 (상위 5건):")
    for order_id, error_count in error_counts[:5]:
        print(f"  • 주문 ID {order_id}: {error_count}개 오류")

# =============================================================================
# 실행 코드
# =============================================================================

def run_comprehensive_validation():
    """포괄적인 주문 데이터 검증 실행"""
    print("=" * 80)
    print("🔍 포괄적인 주문 데이터 검증 시작")
    print("=" * 80)
    
    # 검증 실행
    invalid_orders = validate_orders_with_dask(
        order_seed=order_seed,
        cart_seed=cart_seed,
        product_seed=product,
        batch_size=2000  # 배치 크기 조정 가능
    )
    
    # 상세 분석
    analyze_validation_results(invalid_orders)
    
    return invalid_orders

# 실행
invalid_orders = run_comprehensive_validation()

🔍 포괄적인 주문 데이터 검증 시작
🔍 Dask로 주문 데이터 검증 시작... (총 50,000개)
📊 참조 데이터 준비 중...
🗂️  카트 데이터 인덱싱 중...


상품조합별 인덱싱: 100%|██████████| 50000/50000 [00:00<00:00, 1139967.17cart/s]


📈 인덱싱 완료: 50000명 사용자, 45076개 상품조합
📊 총 25개 배치로 병렬 검증 처리 (배치 크기: 2,000)


검증 배치 준비: 100%|██████████| 25/25 [02:16<00:00,  5.45s/batch]


🔄 주문 검증 병렬 처리 중...


주문 검증: 100%|██████████| 50000/50000 [35:17<00:00, 23.61orders/s]  


📦 검증 결과 병합 중...


결과 병합: 100%|██████████| 25/25 [00:00<00:00, 521679.60batch/s]


✅ 주문 검증 완료 (2117.41초)
📊 총 주문 건수: 50,000개
❌ 검증 실패 건수: 0개
✅ 검증 성공률: 100.00%
⚡ 검증 속도: 24개/초

🎉 모든 주문이 검증을 통과했습니다!
🎉 분석할 오류가 없습니다!


In [49]:
# # 1) 참조용 데이터 준비 및 인덱스 생성
# # ───────────────────────────────────────────────────────────
# if hasattr(product, "to_dict"):
#     prod_list = product.to_dict('records')
# else:
#     prod_list = product
# price_map = {p['id']: p['discounted_price'] for p in prod_list}

# all_carts = cart_seed

# # datetime 파싱 헬퍼 함수
# def parse_iso8601_to_datetime(dt_str):
#     """ISO8601 문자열을 datetime 객체로 변환"""
#     if isinstance(dt_str, str):
#         return pd.to_datetime(dt_str).to_pydatetime()
#     return dt_str

# # 2) 카트 데이터 인덱싱 (검색 성능 향상)
# # ───────────────────────────────────────────────────────────
# print("카트 데이터 인덱싱 중...")

# # 사용자별 카트 인덱스
# cart_by_user = {}
# for c in all_carts:
#     user_id = str(c['user_id'])
#     if user_id not in cart_by_user:
#         cart_by_user[user_id] = []
#     cart_by_user[user_id].append(c)

# # 상품-수량 조합별 카트 인덱스 (시나리오2용)
# cart_by_items = {}
# for c in all_carts:
#     items_key = tuple(sorted((item['product_id'], item['quantity']) 
#                            for item in c['cart_items']))
#     if items_key not in cart_by_items:
#         cart_by_items[items_key] = []
#     cart_by_items[items_key].append(c)

# print(f"인덱싱 완료: {len(cart_by_user)}명 사용자, {len(cart_by_items)}개 상품조합")

# # 3) 최적화된 검증
# # ───────────────────────────────────────────────────────────
# print("주문 검증 시작...")
# invalid = []
# valid_statuses = {"pending_paid", "paid", "shipping", "shipped", "cancelled"}  # pending_paid 추가


# for i, o in enumerate(order_seed):
#     if i % 1000 == 0:  # 진행률 표시
#         print(f"  진행률: {i}/{len(order_seed)} ({i/len(order_seed)*100:.1f}%)")
    
#     errs = []
#     uid = str(o['user_id'])
#     o_ct = parse_iso8601_to_datetime(o['created_at'])

#     # 2-a) status 유효성
#     if o['status'] not in valid_statuses:
#         errs.append(f"invalid status: {o['status']}")

#     # 2-b) 가격·total_price 검증
#     calc_total = 0
#     for it in o['order_items']:
#         pid, qty, pr = it['product_id'], it['quantity'], it['discounted_price']
#         if pid not in price_map:
#             errs.append(f"unknown product_id: {pid}")
#         elif price_map[pid] != pr:
#             errs.append(f"wrong price for {pid}: got {pr}, expect {price_map[pid]}")
#         calc_total += qty * pr
#     if o['total_price'] != calc_total:
#         errs.append(f"total_price {o['total_price']} != computed {calc_total}")

#     # 2-c) 시나리오① 검증 (최적화: 다른 사용자의 카트만 탐색)
#     scenario1_ok = False
#     o_pids = {itm['product_id'] for itm in o['order_items']}
    
#     # 다른 사용자들의 카트만 확인
#     for other_uid, other_carts in cart_by_user.items():
#         if other_uid == uid:
#             continue
        
#         for c in other_carts:
#             ct0 = parse_iso8601_to_datetime(c['created_at'])
#             if not (ct0 <= o_ct <= ct0 + timedelta(days=7)):
#                 continue
            
#             cart_pids = {itm['product_id'] for itm in c['cart_items']}
#             if cart_pids & o_pids:  # 교집합이 있으면 skip
#                 continue
#             if len(c['cart_items']) != len(o['order_items']):
#                 continue
            
#             scenario1_ok = True
#             break
        
#         if scenario1_ok:
#             break

#     # 2-d) 시나리오② 검증 (최적화: 동일한 상품조합만 탐색)
#     scenario2_ok = False
#     order_items_key = tuple(sorted((itm['product_id'], itm['quantity']) 
#                                  for itm in o['order_items']))
    
#     # 동일한 상품조합을 가진 카트들만 확인
#     if order_items_key in cart_by_items:
#         for c in cart_by_items[order_items_key]:
#             ct0 = parse_iso8601_to_datetime(c['created_at'])
#             # 시간 조건만 확인하면 됨 (상품조합은 이미 일치)
#             if o_ct < ct0 or o_ct >= ct0:  # 2-1 또는 2-2 케이스
#                 scenario2_ok = True
#                 break

#     # 둘 중 하나도 만족하지 않으면 invalid
#     if not (scenario1_ok or scenario2_ok):
#         errs.append("no matching scenario")

#     if errs:
#         invalid.append({'order_id': o['id'], 'errors': errs})

# # 4) 결과 출력
# print(f"\n총 주문 건수: {len(order_seed)}")
# print(f"검증 실패 건수: {len(invalid)}")
# if invalid:
#     print("예시 10건:")
#     for rec in invalid[:10]:
#         print(rec)

## 5. 더미 데이터 저장

In [50]:
user_df = pd.DataFrame(user_seed)
# promotion_df = pd.DataFrame(promotion_seed)
cart_df = pd.DataFrame(cart_seed)
order_df = pd.DataFrame(order_seed)

In [51]:
print(user_df.shape)
print(cart_df.shape)
print(order_df.shape)

(50000, 9)
(50000, 5)
(50000, 10)


In [52]:
cart_df.head(2)

,id,user_id,cart_items,created_at,updated_at
0,1,a2ed8ab6-4f78-4534-9214-95a189223c49,"[{'product_id': 4538543, 'quantity': 1, 'disco...",2025-05-08T17:54:25.226284Z,2025-05-10T17:54:25.226284Z
1,2,c97e56ca-6ff9-48a8-96b3-ca9d6fb4136b,"[{'product_id': 4157330, 'quantity': 1, 'disco...",2025-02-14T08:17:18.207795Z,2025-02-15T08:17:18.207795Z


In [53]:
order_df.head(2)

,id,status,tid,payment_status,is_from_cart,user_id,order_items,total_price,created_at,updated_at
0,1,cancelled,TMA2I6EI2ANSXQ7WUJA3,cancelled_kakao,True,046fcbf3-b289-413e-92ea-cfe6e1502080,"[{'product_id': 4795364, 'quantity': 1, 'disco...",439000,2025-04-17T13:08:44.721798Z,2025-04-23T12:55:05.721798Z
1,2,shipped,T8R2GRR0KU75U3VZ7JWI,paid_kakao,True,8a8c669f-d6c8-4561-a386-02d896ed6033,"[{'product_id': 3031030, 'quantity': 1, 'disco...",312890,2025-06-10T09:51:18.568635Z,2025-06-12T18:56:52.568635Z


In [54]:
# os.mkdir("data/generated_data")

In [55]:
# promotion_df = pd.read_csv("data/generated_data/promotion.csv")
# user_df = pd.read_csv("data/generated_data/user.csv")
# cart_df = pd.read_json("data/generated_data/cart.json")
# order_df = pd.read_json("data/generated_data/order.json")

In [56]:
# 1) records 형태로, 한 줄씩(JSONL) 저장
# user 데이터
user_df.to_json(
    'data/generated_data/user_iso.json',
    orient='records',    # 행 단위 JSON 객체
    lines=True,          # 각 객체를 줄바꿈으로 구분 (JSONL)
    force_ascii=False    # 한글 깨짐 방지
)

# # promotion 데이터
# promotion_df.to_json(
#     'data/generated_data/promotion.json',
#     orient='records',    # 행 단위 JSON 객체
#     lines=True,          # 각 객체를 줄바꿈으로 구분 (JSONL)
#     force_ascii=False    # 한글 깨짐 방지
# )

# cart 데이터
cart_df.to_json(
    'data/generated_data/cart_iso.json',
    orient='records',    # 행 단위 JSON 객체
    lines=True,          # 각 객체를 줄바꿈으로 구분 (JSONL)
    force_ascii=False    # 한글 깨짐 방지
)

# order 데이터
order_df.to_json(
    'data/generated_data/order_iso.json',
    orient='records',    # 행 단위 JSON 객체
    lines=True,          # 각 객체를 줄바꿈으로 구분 (JSONL)
    force_ascii=False    # 한글 깨짐 방지
)

In [57]:
# product, brand 데이터는 이미 ISO8601 형태로 변환되어 있으므로, 그대로 저장합니다.
product_iso.to_json(
    'data/generated_data/product_iso.json',
    orient='records',
    lines=True,
    force_ascii=False
)

brand_iso.to_json(
    'data/generated_data/brand_iso.json',
    orient='records',
    lines=True,
    force_ascii=False
)

In [58]:
from collections import Counter

# ID별 등장 횟수 세기
counter = Counter(pd.DataFrame(user_seed)["id"])

# 중복된 ID만 필터링
duplicates = [id_ for id_, count in counter.items() if count > 1]

if duplicates:
    print(f"중복된 ID 목록 ({len(duplicates)}개):")
    for dup in duplicates:
        print(f"  – {dup} (등장 횟수: {counter[dup]})")
else:
    print("중복된 ID가 없습니다.")

중복된 ID가 없습니다.


In [4]:
brand_iso = pd.read_json("data/generated_data/brand_iso.json", lines=True)
product_iso = pd.read_json("data/generated_data/product_iso.json", lines=True)

In [5]:
# category_code를 문자열로 변환하고 4자리인 경우 앞에 "00" 붙이기
def format_category_code(code):
    """
    category_code를 문자열로 변환하고 4자리인 경우 앞에 "00"을 붙임
    """
    # NaN이나 None 값 처리
    if pd.isna(code):
        return code
    
    # 문자열로 변환
    code_str = str(code)
    
    # 4자리인 경우 앞에 "00" 붙이기
    if len(code_str) == 4:
        return "00" + code_str
    else:
        return code_str

# category_code 변환 적용
product_iso['category_code'] = product_iso['category_code'].apply(format_category_code)

# 결과 확인
print("변환 후 category_code 샘플:")
print(product_iso[['id', 'category_code']].head(10))

# 데이터 타입 확인
print(f"\ncategory_code 데이터 타입: {product_iso['category_code'].dtype}")

# 길이별 분포 확인
print(f"\ncategory_code 길이별 분포:")
length_dist = product_iso['category_code'].apply(lambda x: len(str(x)) if pd.notna(x) else 'NaN').value_counts()
print(length_dist)

변환 후 category_code 샘플:
        id category_code
0  2353497        002019
1  3790860        002001
2  3813327        103002
3  4651434        002020
4  3793565        002006
5  1627892        003007
6  3051675        003002
7  4891343        002024
8  3276429        103001
9  3469945        002023

category_code 데이터 타입: object

category_code 길이별 분포:
category_code
6    13798
Name: count, dtype: int64


In [6]:

# category_code의 길이별 분포 확인
def check_category_code_length():
    """
    category_code의 길이를 확인하고 6자리가 아닌 것들을 찾음
    """
    print("=== category_code 길이 검증 ===")
    
    # 1. 전체 길이별 분포
    length_dist = product_iso['category_code'].apply(
        lambda x: len(str(x)) if pd.notna(x) else 'NaN'
    ).value_counts().sort_index()
    
    print("길이별 분포:")
    for length, count in length_dist.items():
        print(f"  {length}자리: {count:,}개")
    
    # 2. 6자리가 아닌 값들 찾기
    non_six_digit = product_iso[
        product_iso['category_code'].apply(
            lambda x: len(str(x)) != 6 if pd.notna(x) else True
        )
    ]
    
    print(f"\n6자리가 아닌 category_code:")
    print(f"총 {len(non_six_digit):,}개 발견")
    
    if len(non_six_digit) > 0:
        print("\n샘플 데이터 (상위 20개):")
        sample_cols = ['id', 'name', 'category_code']
        print(non_six_digit[sample_cols].head(20).to_string(index=False))
        
        # 3. NaN 값 확인
        nan_count = product_iso['category_code'].isna().sum()
        print(f"\nNaN 값: {nan_count:,}개")
        
        # 4. 빈 문자열 확인
        empty_string_count = (product_iso['category_code'] == '').sum()
        print(f"빈 문자열 ('') 값: {empty_string_count:,}개")
        
        # 5. 길이별 상세 분석
        print(f"\n6자리가 아닌 값들의 상세 분석:")
        for length, group in non_six_digit.groupby(
            non_six_digit['category_code'].apply(
                lambda x: len(str(x)) if pd.notna(x) else 'NaN'
            )
        ):
            print(f"  {length}자리: {len(group):,}개")
            if length != 'NaN' and len(group) <= 10:
                unique_values = group['category_code'].unique()
                print(f"    예시 값: {list(unique_values)}")
    else:
        print("✅ 모든 category_code가 6자리입니다!")
    
    return non_six_digit

# 검증 실행
non_six_digit_data = check_category_code_length()

=== category_code 길이 검증 ===
길이별 분포:
  6자리: 13,798개

6자리가 아닌 category_code:
총 0개 발견
✅ 모든 category_code가 6자리입니다!


In [9]:
# 방법 1: 수동으로 UTC ISO8601 변환 후 저장 (추천)
datetime_cols = ['created_at', 'updated_at']

# product 데이터 저장
# product_iso = product.copy()
for col in ['created_at', 'updated_at']:
    if col in product_iso.columns:
        product_iso[col] = product_iso[col].apply(convert_to_iso8601_with_utc)

In [10]:
product_iso

,id,category_code,major_category,sub_category,brand_id,gender,name,discount,discounted_price,price,rank,like_count,view_count,purchase_count,created_at,updated_at,img_url,name_original
0,2353497,002019,outer,anoraks,379,U,LO209_러클닝럽 랙트 켓자_이네비,54,59000,129000,1,66779,61399,16070,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,OL209_러닝클럽 트랙 자켓_네이비
1,3790860,002001,outer,blousons_ma_1,43,M,릴스렉드 미멀니 킷재 [랙블],50,44990,89900,1,9623,33280,7058,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,릴렉스드 미니멀 재킷 [블랙]
2,3813327,103002,shoes,boots,199,U,810ss 케르마 디모 Cchoaral,0,115000,115000,1,9546,39301,5979,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,810s 마르케 모디 Charcoal
3,4651434,002020,outer,cardigans,354,F,즈먼우 드리브 림슬 쿱스 넥넥 지저 가건디 [랙블],50,14890,29900,1,1208,10079,0,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,우먼즈 리브드 슬림 스쿱 넥 저지 가디건 [블랙]
4,3793565,002006,outer,coach_jackets,43,M,티시 저레 어시 윈브이레드커 킷재 [랙블],22,38890,49900,1,11885,95364,9641,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,시티 레저 시어 윈드브레이커 재킷 [블랙]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13793,4950401,001001,tops,t_shirts,380,U,IAS 셜에센 플와 리넥헨 팔반 츠셔티_피먼그트 콜차,10,29900,33300,300,1214,26017,0,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,ASI 에센셜 와플 헨리넥 반팔 티셔츠_피그먼트 차콜
13794,4679992,002018,outer,training_jackets,47,M,ENCSSEE 2.0 (센스에 2.0) 트이레닝 트슈_Balck PYU25104ZZ1,0,129000,129000,300,11,453,0,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,ESSENCE 2.0 (에센스 2.0) 트레이닝 슈트_Black YUP25104Z1
13795,1350363,003004,pants,training_pants,834,F,링스트 딩밴 켓포 츠팬 [랙블],50,38000,76000,300,17573,7472,11216,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,스트링 밴딩 포켓 팬츠 [블랙]
13796,4787523,002017,outer,trucker_jackets,1267,M,[SIH] 턴헤링 퍼점 다그크린,20,207200,259000,300,24,990,0,2025-05-17T10:32:18Z,2025-05-17T10:32:18Z,https://highfive-goorm-images.s3.amazonaws.com...,[HIS] 헤링턴 점퍼 다크그린


In [11]:
brand_iso

,id,brand_kor,brand_eng,like_count,created_at,updated_at,brand_kor_original,brand_eng_original
0,1,토토앤,twtnoae,21858,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,토앤토,tawntoe
1,2,스쳐케스,skeehrcs,9279,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,스케쳐스,skechers
2,3,아클키래식,aiiilicakssc,34177,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,아키클래식,akiiiclassic
3,4,츄스바코,csuhbaco,63950,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,츄바스코,chubasco
4,5,스우포,ooofs,16418,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,우포스,oofos
...,...,...,...,...,...,...,...,...
1673,1674,착침맨,cimcmhaahkn,10256,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,침착맨,chimchakman
1674,1675,낙디그12,dnydbagk,13984,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,디그낙12,dbydgnak
1675,1676,스우데 스엑 키마나,dihseuanxecma,9809,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,데우스 엑스 마키나,deusexmachina
1676,1677,로산,srano,19689,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,산로,sanro


In [83]:
product_iso.to_json(
    'data/generated_data/product_iso.json',
    orient='records',
    lines=True,
    force_ascii=False
)

In [63]:
brand_iso[brand_iso["brand_kor_original"] == "무신사 스탠다드"]

,id,brand_kor,brand_eng,like_count,created_at,updated_at,brand_kor_original,brand_eng_original
42,43,신무사 스다탠드,maarsanistusdnd,922514,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,무신사 스탠다드,musinsastandard


In [64]:
brand_iso[brand_iso["brand_kor_original"] == "나이키"]

,id,brand_kor,brand_eng,like_count,created_at,updated_at,brand_kor_original,brand_eng_original
14,15,이키나,nkie,405816,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,나이키,nike


In [65]:
brand_iso[brand_iso["brand_kor_original"] == "스파오"]

,id,brand_kor,brand_eng,like_count,created_at,updated_at,brand_kor_original,brand_eng_original
43,44,파스오,sapo,218474,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,스파오,spao


In [66]:
brand_iso.sort_values(by="like_count", ascending=False).head(20)

,id,brand_kor,brand_eng,like_count,created_at,updated_at,brand_kor_original,brand_eng_original
42,43,신무사 스다탠드,maarsanistusdnd,922514,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,무신사 스탠다드,musinsastandard
12,13,아다디스,aiadds,720347,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,아디다스,adidas
14,15,이키나,nkie,405816,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,나이키,nike
50,51,낫커버,caonvert,385539,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,커버낫,covernat
343,344,디이즈버네스댓,thieervsshnaitt,374850,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,디스이즈네버댓,thisisneverthat
94,95,일예,ylae,329420,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,예일,yale
18,19,뉴란발스,nawbclenae,325480,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,뉴발란스,newbalance
340,341,코그드라피,cepagrhdoy,320219,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,코드그라피,codegraphy
178,179,와윌키리,wklyawlicy,299047,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,와키윌리,wackywilly
45,46,스반,vnas,291483,2025-05-17 10:32:18+00:00,2025-05-17 10:32:18+00:00,반스,vans


## 6. Promotion 데이터 생성

In [22]:
import json
import pandas as pd
from datetime import datetime, timedelta
import random

def create_promotion_data(brand_df, start_base_date = datetime(2024, 1, 1)):
    # like_count 기준으로 내림차순 정렬하여 상위 20개 브랜드 선택
    sorted_brands = brand_df.sort_values('like_count', ascending=False)
    top_20_brands = sorted_brands.head(20)
    
    promotions = []
    end_target_date = datetime(2025, 7, 31)
    total_days = (end_target_date - start_base_date).days
    
    # 20개 프로모션을 시간순으로 배치 (그룹 나누지 않음)
    for i in range(20):
        # top 20 브랜드에서 랜덤으로 선택 (중복 허용)
        brand = top_20_brands.sample(n=1).iloc[0]
        
        # 전체 기간을 20개로 나누어 시작점 분산
        start_offset = (total_days // 20) * i + random.randint(-10, 10)
        promotion_start = start_base_date + timedelta(days=start_offset)
        
        # 프로모션 기간: 30-60일
        duration_days = random.randint(30, 60)
        promotion_end = promotion_start + timedelta(days=duration_days)
        
        # 8월 말을 넘지 않도록 조정
        if promotion_end > end_target_date:
            promotion_end = end_target_date
            promotion_start = promotion_end - timedelta(days=duration_days)
            if promotion_start < start_base_date:
                promotion_start = start_base_date
        
        current_time = datetime.now()
        
        promotion = {
            "title": f"{brand['brand_kor']} {get_promotion_title_suffix(i)}",
            "description": f"{brand['brand_kor']}에서 진행하는 특별 프로모션입니다. 놓치지 마세요!",
            "brand_id": int(brand['id']),
            "banner_image_url": f"https://cdn.example.com/banners/promotion_{brand['id']}_{i+1}.jpg",
            "is_active": False,
            "start_date": promotion_start.strftime('%Y-%m-%dT%H:%M:%SZ'),  # 마이크로초 제거
            "end_date": promotion_end.strftime('%Y-%m-%dT%H:%M:%SZ'),      # 마이크로초 제거
            "created_at": current_time.strftime('%Y-%m-%dT%H:%M:%SZ'),     # 마이크로초 제거
            "updated_at": current_time.strftime('%Y-%m-%dT%H:%M:%SZ'),     # 마이크로초 제거
            "destination_url": f"/brand/{int(brand['id'])}"
        }
        
        promotions.append(promotion)
    
    # 시작일 기준으로 정렬
    promotions.sort(key=lambda x: x['start_date'])
    
    # 마지막 3개 프로모션 활성화
    for i in range(len(promotions) - 3, len(promotions)):
        promotions[i]['is_active'] = True
    
    return promotions

def get_promotion_title_suffix(index):
    """프로모션 제목 접미사 생성"""
    titles = [
        "특별 할인 이벤트", "여름 특가 프로모션", 
        "신제품 런칭 이벤트", "주년 기념 할인", "VIP 고객 특가",
        "플래시 세일", "신규 고객 웰컴 할인", "리뉴얼 기념 특가",
        "한정 시간 할인", "멤버십 특별 혜택", "브랜드 데이 이벤트",
        "고객 감사 특가",  "스페셜 오퍼"
    ]
    return titles[index % len(titles)]

def show_promotion_overlap_info(promotions):
    """프로모션 겹침 정보 출력"""
    print("=== 프로모션 기간 겹침 현황 ===")
    
    for i, promo in enumerate(promotions):
        start = datetime.strptime(promo['start_date'], '%Y-%m-%dT%H:%M:%SZ')  # 수정된 형식
        end = datetime.strptime(promo['end_date'], '%Y-%m-%dT%H:%M:%SZ')      # 수정된 형식
        
        overlapping_brands = []
        for j, other_promo in enumerate(promotions):
            if i != j:
                other_start = datetime.strptime(other_promo['start_date'], '%Y-%m-%dT%H:%M:%SZ')
                other_end = datetime.strptime(other_promo['end_date'], '%Y-%m-%dT%H:%M:%SZ')
                
                # 기간이 겹치는지 확인
                if max(start, other_start) < min(end, other_end):
                    brand_name = other_promo['title'].split()[0]
                    overlapping_brands.append(brand_name)
        
        current_brand = promo['title'].split()[0]
        duration = (end - start).days
        active_status = "🟢 활성" if promo['is_active'] else "🔴 비활성"
        print(f"{current_brand} 프로모션: {promo['start_date'][:10]} ~ {promo['end_date'][:10]} ({duration}일) {active_status}")
        print(f"  겹치는 브랜드: {overlapping_brands if overlapping_brands else '없음'} (총 {len(overlapping_brands)}개)")
        print()

def print_total_promotion_period(promotions):
    """전체 프로모션 기간 출력"""
    if not promotions:
        return
    
    start_dates = [datetime.strptime(p['start_date'], '%Y-%m-%dT%H:%M:%SZ') for p in promotions]
    end_dates = [datetime.strptime(p['end_date'], '%Y-%m-%dT%H:%M:%SZ') for p in promotions]
    
    total_start = min(start_dates)
    total_end = max(end_dates)
    total_duration = (total_end - total_start).days
    
    # 활성 프로모션 정보
    active_promotions = [p for p in promotions if p['is_active']]
    
    print("=== 전체 프로모션 기간 ===")
    print(f"시작일: {total_start.strftime('%Y-%m-%d')}")
    print(f"종료일: {total_end.strftime('%Y-%m-%d')}")
    print(f"총 기간: {total_duration}일")
    print(f"활성 프로모션: {len(active_promotions)}개")
    print()

# 프로모션 데이터 생성
promotion_data = create_promotion_data(brand_iso, datetime(2024, 1, 1))

print(f"총 {len(promotion_data)}개의 프로모션이 생성되었습니다.")

# 전체 프로모션 기간 출력
print_total_promotion_period(promotion_data)

# 프로모션 겹침 정보 출력
show_promotion_overlap_info(promotion_data)

# 브랜드별 프로모션 개수 확인
brand_counts = {}
for promo in promotion_data:
    brand_id = promo['brand_id']
    brand_name = promo['title'].split()[0]
    brand_counts[brand_name] = brand_counts.get(brand_name, 0) + 1

print("=== 브랜드별 프로모션 개수 ===")
for brand, count in sorted(brand_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{brand}: {count}개")

총 20개의 프로모션이 생성되었습니다.
=== 전체 프로모션 기간 ===
시작일: 2024-01-01
종료일: 2025-07-31
총 기간: 577일
활성 프로모션: 3개

=== 프로모션 기간 겹침 현황 ===
필이루네미트 프로모션: 2024-01-01 ~ 2024-02-27 (57일) 🔴 비활성
  겹치는 브랜드: ['뉴란발스', '아웃탠스딩'] (총 2개)

뉴란발스 프로모션: 2024-01-23 ~ 2024-02-22 (30일) 🔴 비활성
  겹치는 브랜드: ['필이루네미트'] (총 1개)

아웃탠스딩 프로모션: 2024-02-23 ~ 2024-04-02 (39일) 🔴 비활성
  겹치는 브랜드: ['필이루네미트', '버스컨'] (총 2개)

버스컨 프로모션: 2024-03-17 ~ 2024-04-30 (44일) 🔴 비활성
  겹치는 브랜드: ['아웃탠스딩', '내래오셔널그지픽'] (총 2개)

내래오셔널그지픽 프로모션: 2024-04-14 ~ 2024-06-11 (58일) 🔴 비활성
  겹치는 브랜드: ['버스컨', '어드반레스'] (총 2개)

어드반레스 프로모션: 2024-05-29 ~ 2024-07-23 (55일) 🔴 비활성
  겹치는 브랜드: ['내래오셔널그지픽', '어드반레스', '필이루네미트'] (총 3개)

어드반레스 프로모션: 2024-06-24 ~ 2024-08-04 (41일) 🔴 비활성
  겹치는 브랜드: ['어드반레스', '필이루네미트'] (총 2개)

필이루네미트 프로모션: 2024-07-18 ~ 2024-09-06 (50일) 🔴 비활성
  겹치는 브랜드: ['어드반레스', '어드반레스', '일예'] (총 3개)

일예 프로모션: 2024-08-09 ~ 2024-09-23 (45일) 🔴 비활성
  겹치는 브랜드: ['필이루네미트', '아다디스'] (총 2개)

아다디스 프로모션: 2024-09-18 ~ 2024-10-21 (33일) 🔴 비활성
  겹치는 브랜드: ['일예', '아다디스'] (총 2개)

아다디스 프로모션: 2024-

In [23]:
promotion_data

[{'title': '필이루네미트 특별 할인 이벤트',
  'description': '필이루네미트에서 진행하는 특별 프로모션입니다. 놓치지 마세요!',
  'brand_id': 206,
  'banner_image_url': 'https://cdn.example.com/banners/promotion_206_1.jpg',
  'is_active': False,
  'start_date': '2024-01-01T00:00:00Z',
  'end_date': '2024-02-27T00:00:00Z',
  'created_at': '2025-06-16T22:06:10Z',
  'updated_at': '2025-06-16T22:06:10Z',
  'destination_url': '/brand/206'},
 {'title': '뉴란발스 여름 특가 프로모션',
  'description': '뉴란발스에서 진행하는 특별 프로모션입니다. 놓치지 마세요!',
  'brand_id': 19,
  'banner_image_url': 'https://cdn.example.com/banners/promotion_19_2.jpg',
  'is_active': False,
  'start_date': '2024-01-23T00:00:00Z',
  'end_date': '2024-02-22T00:00:00Z',
  'created_at': '2025-06-16T22:06:10Z',
  'updated_at': '2025-06-16T22:06:10Z',
  'destination_url': '/brand/19'},
 {'title': '아웃탠스딩 신제품 런칭 이벤트',
  'description': '아웃탠스딩에서 진행하는 특별 프로모션입니다. 놓치지 마세요!',
  'brand_id': 454,
  'banner_image_url': 'https://cdn.example.com/banners/promotion_454_3.jpg',
  'is_active': False,
  'star

In [24]:
pd.DataFrame(promotion_data).to_json(
    'data/generated_data/promotion_iso.json',
    orient='records',    # 행 단위 JSON 객체
    lines=True,          # 각 객체를 줄바꿈으로 구분 (JSONL)
    force_ascii=False    # 한글 깨짐 방지
)